<a href="https://colab.research.google.com/github/Krish9676/Crop-Growth-stage-prediction/blob/main/Prod_New_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio
!pip install sentinelhub
!pip install flask_limiter
!pip install flask
!pip install earthpy
!pip install geojson
!pip install pyproj

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.1 MB/s eta 0:00:00


In [ ]:
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import warnings
import rasterio
import requests
import tempfile
import pyproj
from io import BytesIO
from rasterio import features
from shapely.geometry import Polygon
from pyproj import CRS
from sentinelhub import SHConfig, SentinelHubRequest, DataCollection, MimeType, BBox, bbox_to_dimensions
from collections import defaultdict
from shapely.geometry import Point
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.ensemble import RandomForestRegressor
from rasterio.io import MemoryFile
from rasterio.enums import Compression
from shapely.geometry import shape, mapping
from rasterio.features import shapes
from rasterio.transform import from_bounds
from rasterio.transform import from_origin
import rasterio.features
from shapely.geometry import box
import re
import logging
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy.signal import savgol_filter
from datetime import datetime, timedelta
from pyproj import CRS, Transformer
import base64
from rasterio.transform import xy
from typing import Dict, Union

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

#################################################### Read credentials and load Farm Boundary##############################

def read_credentials(file_path):
    """
    Reads Sentinel Hub credentials from a file.

    Parameters:
    file_path (str): Path to the credentials file.

    Returns:
    tuple: A tuple containing instance_id, client_id, and client_secret.
    """
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()
            instance_id = lines[0].split('=')[1].strip().strip('"')
            client_id = lines[1].split('=')[1].strip().strip('"')
            client_secret = lines[2].split('=')[1].strip().strip('"')
        return instance_id, client_id, client_secret
    except Exception as e:
        logging.error(f"Failed to read credentials: {e}")
        raise

def setup_config(file_path):
    """
    Sets up the Sentinel Hub configuration.

    Parameters:
    file_path (str): Path to the credentials file.

    Returns:
    SHConfig: The Sentinel Hub configuration object.
    """
    instance_id, client_id, client_secret = read_credentials(file_path)
    config = SHConfig()
    config.instance_id = instance_id
    config.sh_client_id = client_id
    config.sh_client_secret = client_secret

    if not (instance_id and client_id and client_secret):
        raise ValueError("Sentinel Hub credentials are not set properly.")

    return config

def get_user_coordinates_and_sowing_date(mobile, farm_name):
    """
    Retrieves user coordinates and sowing date from an API.

    Parameters:
    mobile (str): User's mobile number.
    farm_name (str): The farm's name.

    Returns:
    tuple: A tuple containing a GeoDataFrame with the polygon, a bounding box, and the crop type.
    """
    url = f"https://3mpqory1se.execute-api.ap-south-1.amazonaws.com/prod/getFarm?mobile={mobile}&farm_name={farm_name}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        if not data:
            logging.warning("No data returned from the API.")
            return gpd.GeoDataFrame(), [], None

        # Process the first record in the data
        farm_data = data[0]
        coordinates = farm_data.get('co-ordinates', [])
        if not coordinates:
            logging.warning("Coordinates are missing in the API response.")
            return gpd.GeoDataFrame(), [], None

        # Extract latitudes and longitudes
        lats = [coord['latitude'] for coord in coordinates]
        lngs = [coord['longitude'] for coord in coordinates]

        # Create a polygon from coordinates
        polygon_geom = Polygon(zip(lngs, lats))

        # Extract additional data
        sowing_date = farm_data.get('sowing_date')
        crop_type = farm_data.get('crop')
        country = farm_data.get('country')
        Season_satrt = farm_data.get ("start_of_season")
        attributes = {
            'date': sowing_date,
            'crop_type': crop_type,
            'mobile_no': str(mobile),
            'farm_name': farm_name,
            'country': country,
            'Season_satrt': Season_satrt,
            'field_id': 1
        }

        # Create a GeoDataFrame
        polygon = gpd.GeoDataFrame(pd.DataFrame(attributes, index=[0]), crs='EPSG:4326', geometry=[polygon_geom])

        # Compute bounding box
        bounding_box = polygon.total_bounds.tolist()  # [minx, miny, maxx, maxy]

        return polygon, bounding_box, crop_type, country, Season_satrt

    except requests.RequestException as e:
        logging.error(f"Failed to retrieve farm data: {e}")
    except KeyError as e:
        logging.error(f"Missing expected key in API response: {e}")
    except Exception as e:
        logging.error(f"Unexpected error: {e}")

    return gpd.GeoDataFrame(), [], None, None, None

CROP_DURATIONS = {
    "India": {
        "Rice": {"pre_days": 40,"harvest_days": 140,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Wheat": {"pre_days": 40,"harvest_days": 160,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [10, 11, 12, 1],"summer": [2, 3, 4, 5]}},
        "Potato": {"pre_days": 30,"harvest_days": 140,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [10, 11, 12, 1],"summer": [2, 3, 4, 5]}},
        "Mustard": {"pre_days": 30,"harvest_days": 160,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Sanghar_White_Onion": {"pre_days": 10,"harvest_days": 200,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "White_Onion": {"pre_days": 30,"harvest_days": 160,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Red_Onion": {"pre_days": 30,"harvest_days": 160,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Maize": {"pre_days": 30,"harvest_days": 150,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Cotton": {"pre_days": 40,"harvest_days": 180,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Gram": {"pre_days": 30,"harvest_days": 140,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Soyabean": {"pre_days": 30,"harvest_days": 140,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Barley": {"pre_days": 30,"harvest_days": 150,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Sugarcane": {"pre_days": 30,"harvest_days": 365,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Banana": {"pre_days": 50,"harvest_days": 330,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}}
    },
    # New countries from provided data
    "Slovakia": {
        "Rice": {"pre_days": 60,"harvest_days": 230,"seasons": {"Summer": [4, 5, 6, 7]}},
        "Wheat": {"pre_days": 60,"harvest_days": 270,"seasons": {"Rabi": [9, 10, 11, 12],"summer": [3, 4]}},
        "Maize": {"pre_days": 60,"harvest_days": 250,"seasons": {"summer": [2, 3, 4, 5]}},
        "Soyabean": {"pre_days": 40,"harvest_days": 156,"seasons": {"summer": [3, 4, 5, 6, 7]}},
        "Barley": {"pre_days": 40,"harvest_days": 150,"seasons": {"Rabi": [5, 6, 7, 8],"summer": [2, 3, 4]}}
    },
    "Philippines": {
        "Rice": {"pre_days": 40,"harvest_days": 160,"seasons": {"Kharif": [5, 6, 7, 8, 9, 10],"Rabi": [11, 12, 1, 2, 3, 4]}},
        "Maize": {"pre_days": 20,"harvest_days": 140,"seasons": {"Kharif": [5, 6, 7, 8, 9],"Rabi": [11, 12, 1, 2]}}
    },
    "Netherlands": {
        "White_Onion": {"pre_days": 30,"harvest_days": 140,"seasons": {"Summer": [2, 3, 4, 5, 6]}},
    },
    "Egypt": {
        "Potato": {"pre_days": 40,"harvest_days": 170,"seasons": {"Rabi": [9, 10, 11, 12],"Summer": [1, 2, 3, 4]}},
        "Wheat": {"pre_days": 40,"harvest_days": 170,"seasons": {"Rabi": [9, 10, 11, 12]}},
        "Rice": {"pre_days": 40,"harvest_days": 170,"seasons": {"Summer": [1, 2, 3, 4]}},
        "Maize": {"pre_days": 40,"harvest_days": 150,"seasons": {"Summer": [1, 2, 3, 4]}}
    },
    # Default values for any crop/country not specified
    "Default": {
        "Rice": {"pre_days": 30,"harvest_days": 160,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Wheat": {"pre_days": 30,"harvest_days": 170,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Potato": {"pre_days": 20,"harvest_days": 150,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Mustard": {"pre_days": 30,"harvest_days": 180,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "White_Onion": {"pre_days": 30,"harvest_days": 160,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Red_Onion": {"pre_days": 30,"harvest_days": 160,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Maize": {"pre_days": 30,"harvest_days": 150,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Cotton": {"pre_days": 40,"harvest_days": 180,"seasons":{"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Gram": {"pre_days": 30,"harvest_days": 140,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Soyabean": {"pre_days": 30,"harvest_days": 140,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Barley": {"pre_days": 30,"harvest_days": 160,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}},
        "Default": {"pre_days": 30,"harvest_days": 180,"seasons": {"Kharif": [6, 7, 8, 9],"Rabi": [11, 12, 1 ],"summer": [2, 3, 4, 5]}}
    }}

def assign_crop_season(polygon, country):
    """
    Assigns the crop season based on the sowing month and country.

    Parameters:
    polygon (GeoDataFrame): A GeoDataFrame containing the sowing date.
    country (str): The country for which crop seasons are being determined.

    Returns:
    GeoDataFrame: The input GeoDataFrame with an added 'crop_season' column.
    """
    polygon['date'] = pd.to_datetime(polygon['date'], format='mixed')

    # Get country-specific seasons or default if not found
    country_data = CROP_DURATIONS.get(country, CROP_DURATIONS["Default"])
    crop_type = polygon.loc[0, 'crop_type'] if 'crop_type' in polygon.columns else "Default"

    # Handle the case when crop_type is not in the country's data
    if crop_type not in country_data:
        # Try to get the crop from Default country
        if crop_type in CROP_DURATIONS["Default"]:
            crop_data = CROP_DURATIONS["Default"][crop_type]
        else:
            # If crop not found in Default, use the Default crop
            crop_data = CROP_DURATIONS["Default"]["Default"]
    else:
        crop_data = country_data[crop_type]

    def determine_season(date):
        month = date.month
        for season, months in crop_data["seasons"].items():
            if month in months:
                return season
        return "Unknown"

    polygon['crop_season'] = polygon['date'].apply(determine_season)
    return polygon

def calculate_time_interval(date, country, crop_type):
    """
    Calculates the start and end dates for a crop season based on country and crop type.

    Parameters:
    date (Timestamp): The sowing date.
    country (str): The country where the farm is located.
    crop_type (str): The type of crop being grown.

    Returns:
    tuple: A tuple containing the start date and the end date.
    """
    today = pd.Timestamp(datetime.now())

    if pd.isna(date):
        logging.error("Invalid date provided.")
        return None, None

    # Get country-specific durations or default if not found
    country_data = CROP_DURATIONS.get(country, CROP_DURATIONS["Default"])

    # Handle the case when crop_type is not in the country's data
    if crop_type not in country_data:
        # Try to get the crop from Default country
        if crop_type in CROP_DURATIONS["Default"]:
            crop_data = CROP_DURATIONS["Default"][crop_type]
        else:
            # If crop not found in Default, use the Default crop
            crop_data = CROP_DURATIONS["Default"]["Default"]
    else:
        crop_data = country_data[crop_type]

    pre_days = crop_data["pre_days"]
    harvest_days = crop_data["harvest_days"]

    start_date = date - timedelta(days=pre_days)
    end_date = date + timedelta(days=harvest_days)

    if start_date <= today <= end_date:
        end_date = today

    return start_date, end_date

def process_farm_data(file_path, mobile, farm_name ):
    """
    Processes farm data by retrieving, validating, and calculating necessary details.

    Parameters:
    file_path (str): Path to the credentials file.
    mobile (str): User's mobile number.
    farm_name (str): The farm's name.
    country (str): The country where the farm is located.

    Returns:
    tuple: A tuple containing start date, end date, GeoDataFrame with polygon, crop season, config, area, and crop type.
    """
    try:
        config = setup_config(file_path)
        polygon, bounding_box, crop_type, country, start_sowing_date = get_user_coordinates_and_sowing_date(mobile, farm_name)
        if polygon.empty:
            logging.warning("No farm data found.")
            return None, None, gpd.GeoDataFrame(), None, None, None, None, None, None

        polygon['crop_type'] = crop_type  # Add crop_type to the polygon dataframe
        polygon = assign_crop_season(polygon, country)
        crop_season = polygon.loc[0, 'crop_season']
        sowing_date = polygon.loc[0, 'date']
        start_date, end_date = calculate_time_interval(sowing_date, country, crop_type)

        if not start_date or not end_date:
            logging.error("Invalid crop season.")
            return None, None, gpd.GeoDataFrame(), None, None, None, None, None, None

        polygon['field_id'] = 1
        polygon = polygon.to_crs(epsg=3857)
        area_sq_meters = polygon.geometry.area[0]
        area = area_sq_meters * 0.000247105  # Convert to acres
        polygon = polygon.to_crs(epsg=4326)

        # Add information about harvest duration to polygon for reference
        country_data = CROP_DURATIONS.get(country, CROP_DURATIONS["Default"])

        # Handle the case when crop_type is not in the country's data
        if crop_type not in country_data:
            # Try to get the crop from Default country
            if crop_type in CROP_DURATIONS["Default"]:
                crop_data = CROP_DURATIONS["Default"][crop_type]
            else:
                # If crop not found in Default, use the Default crop
                crop_data = CROP_DURATIONS["Default"]["Default"]
        else:
            crop_data = country_data[crop_type]

        polygon['harvest_days'] = crop_data["harvest_days"]
        polygon['pre_days'] = crop_data["pre_days"]
        polygon['expected_harvest_date'] = sowing_date + timedelta(days=crop_data["harvest_days"])

        return start_date, end_date, polygon, crop_season, config, area, crop_type, country, start_sowing_date

    except Exception as e:
        logging.error(f"An error occurred during farm data processing: {e}")
        return None, None, gpd.GeoDataFrame(), None, None, None, None, None, None


################################################################Data validation and Downloading #########################################

def sr_to_height(sr_index):
    """Convert SR Index to plant height."""
    sr_index_clamped = np.clip(sr_index, 0.1, 10)  # Clamping SR Index
    return 0.5 * sr_index_clamped + 1.0

def calculate_plant_height(vh_band, vv_band):
    """Calculate plant height from VH and VV polarization bands."""
    sr_index = np.divide(vh_band, vv_band, out=np.full_like(vh_band, np.nan), where=vv_band>1e-6)
    return sr_to_height(sr_index)

def calculate_rvi(vh_band, vv_band):
    """Calculate Radar Vegetation Index (RVI) from VH and VV polarization."""
    denominator = vv_band + vh_band
    # Avoid division by zero or very small values
    rvi = np.divide(4 * vh_band, denominator, out=np.full_like(vh_band, np.nan), where=denominator>1e-6)
    # Clip values to a reasonable range (RVI is typically between 0 and 1)
    return np.clip(rvi, 0, 1.5)


# Alternative: If you want to keep normalization, use this modified normalize function
def normalize_index(data, min_val=0.0, max_val=1.0):
    """Normalize index values to a specified range."""
    if not isinstance(data, np.ndarray) or data.size == 0:
        logging.error("Input data must be a non-empty numpy array.")
        raise ValueError("Invalid input data for normalization.")

    data_min = np.nanmin(data)
    data_max = np.nanmax(data)

    if data_min == data_max:
        logging.warning("Input data has uniform values. Returning normalized data of 0.0.")
        return np.zeros_like(data)

    normalized_data = (data - data_min) / (data_max - data_min)
    # FIXED: Remove the extra scaling that was causing negative values
    return np.clip(normalized_data, min_val, max_val)

def save_data_temporarily(data, file_name, crs=None, transform=None, bounds=None):
    """
    Save data to a temporary TIFF file and return file path and metadata.
    """
    try:
        temp_tiff_dir = tempfile.mkdtemp()
        file_path = os.path.join(temp_tiff_dir, file_name)

        # Set default CRS if not provided
        crs = crs or 'EPSG:4326'
        if transform is None and bounds is not None:
            left, bottom, right, top = bounds
            transform = from_bounds(left, bottom, right, top, data.shape[1], data.shape[0])

        # Write the TIFF file
        with rasterio.open(
            file_path, 'w', driver='GTiff',
            height=data.shape[0], width=data.shape[1], count=1, dtype='float32',
            crs=crs, transform=transform, compress=Compression.lzw
        ) as dst:
            dst.write(data, 1)

        # Read the data back into memory
        with rasterio.open(file_path) as src:
            tiff_data = src.read(1)
            height, width = tiff_data.shape

        # FIXED: Return format matching your working version
        return {
            "path": file_path,
            "data": tiff_data,
            "crs": crs,
            "transform": transform,
            "height": height,
            "width": width
        }
    except Exception as e:
        logging.error(f"Failed to save data temporarily: {e}")
        raise

def reproject_bounds(bbox, src_crs, dst_crs):
    """
    Reproject the bounding box from src_crs to dst_crs.

    Args:
        bbox (sentinelhub.BBox): Bounding box object
        src_crs (str): Source CRS
        dst_crs (str): Destination CRS

    Returns:
        tuple: Reprojected bounds (minx, miny, maxx, maxy)
    """
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True)
    minx, miny = bbox.min_x, bbox.min_y
    maxx, maxy = bbox.max_x, bbox.max_y
    minx, miny = transformer.transform(minx, miny)
    maxx, maxy = transformer.transform(maxx, maxy)
    return (minx, miny, maxx, maxy)

def validate_date_range(time_stamp):
    """
    Validate that the time_stamp is in the correct format.

    Args:
        time_stamp (tuple): A tuple of (start_date, end_date) as ISO strings

    Returns:
        bool: True if valid, False otherwise
    """
    try:
        start_date = datetime.fromisoformat(time_stamp[0])
        end_date = datetime.fromisoformat(time_stamp[1])
        return start_date < end_date
    except (ValueError, TypeError):
        return False

def process_sar_data(farm_name, bbox, time_stamp, date_range, config, max_retries=3):
    """
    Process SAR data and save all relevant SAR-derived TIFF files.

    Args:
        farm_name (str): Name of the farm
        bbox (sentinelhub.BBox): Bounding box of the area of interest
        time_stamp (tuple): Time interval as (start_date, end_date)
        date_range (str): String representation of the date range
        config (sentinelhub.SHConfig): Sentinel Hub configuration
        max_retries (int): Maximum number of retry attempts for API requests

    Returns:
        dict: Dictionary with paths to generated TIFF files and metadata
    """
    if not validate_date_range(time_stamp):
        logging.error(f"Invalid time_stamp format for {date_range}. Expected ISO format dates.")
        return None

    try:
        # Standardize farm name for filenames (remove spaces and special characters)
        safe_farm_name = re.sub(r'[^\w]', '_', farm_name)

        evalscript_sar = """
        //VERSION=3
        function setup() {
            return {
                input: [{bands: ["VH", "VV"]}],
                output: {bands: 2, sampleType: "FLOAT32"}
            };
        }
        function evaluatePixel(samples) {
            return [samples.VH, samples.VV];
        }
        """

        attempts = 0
        sar_data = None

        while attempts < max_retries and sar_data is None:
            try:
                request_sar = SentinelHubRequest(
                    evalscript=evalscript_sar,
                    input_data=[
                        SentinelHubRequest.input_data(
                            data_collection=DataCollection.SENTINEL1_IW,
                            time_interval=time_stamp
                        )
                    ],
                    responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
                    bbox=bbox,
                    size=bbox_to_dimensions(bbox, resolution=10),
                    config=config
                )

                sar_data = request_sar.get_data()
                if not sar_data or len(sar_data) == 0:
                    logging.info(f"No SAR data available for {date_range}.")
                    return None
            except Exception as e:
                attempts += 1
                logging.warning(f"Attempt {attempts} failed: {e}. Retrying...")
                if attempts >= max_retries:
                    logging.error(f"Failed to retrieve SAR data after {max_retries} attempts: {e}")
                    return None
                # Wait with increasing delay between retries
                time.sleep(2 * attempts)

        sar_vh = sar_data[0][:, :, 0]
        sar_vv = sar_data[0][:, :, 1]

        if np.all(np.isnan(sar_vh)) or np.all(np.isnan(sar_vv)):
            logging.info(f"SAR data contains only NaN values for {date_range}.")
            return None

        sar_vvvh = np.divide(sar_vh, sar_vv, out=np.full_like(sar_vh, np.nan), where=sar_vv > 1e-6)
        sar_rvi = calculate_rvi(sar_vh, sar_vv)
        plant_height = sr_to_height(sar_vvvh)

        # Reproject bounding box from EPSG:3857 to EPSG:4326
        bbox_epsg4326 = reproject_bounds(bbox, 'EPSG:3857', 'EPSG:4326')

        # Standardized file names
        prefix = f"FIELD_{safe_farm_name}_{date_range}"
        vv_file_name = f"{prefix}_SAR_VV.tiff"
        vh_file_name = f"{prefix}_SAR_VH.tiff"
        vvvh_file_name = f"{prefix}_SAR_VVVH.tiff"
        rvi_file_name = f"{prefix}_SAR_RVI.tiff"
        height_file_name = f"{prefix}_PLANT_HEIGHT.tiff"

        # Save all arrays as TIFFs
        vv_result = save_data_temporarily(sar_vv, vv_file_name, crs='EPSG:4326', bounds=bbox_epsg4326)
        vh_result = save_data_temporarily(sar_vh, vh_file_name, crs='EPSG:4326', bounds=bbox_epsg4326)
        vvvh_result = save_data_temporarily(sar_vvvh, vvvh_file_name, crs='EPSG:4326', bounds=bbox_epsg4326)
        rvi_result = save_data_temporarily(sar_rvi, rvi_file_name, crs='EPSG:4326', bounds=bbox_epsg4326)
        height_result = save_data_temporarily(plant_height, height_file_name, crs='EPSG:4326', bounds=bbox_epsg4326)

        return {
            "sar_vv": vv_result,
            "sar_vh": vh_result,
            "sar_vvvh": vvvh_result,
            "sar_rvi": rvi_result,
            "plant_height": height_result,
            "date_range": date_range
        }
    except Exception as e:
        logging.error(f"Failed to process SAR data for {date_range}: {e}")
        return None

import time
def process_optical_data(farm_name, bbox, time_stamp, date_range, index_name, config, max_retries=3):
    """
    Process optical data and save index TIFF temporarily.
    CORRECTED VERSION based on your working implementation.
    """
    if not validate_date_range(time_stamp):
        logging.error(f"Invalid time_stamp format for {date_range}. Expected ISO format dates.")
        return None

    try:
        safe_farm_name = re.sub(r'[^\w]', '_', farm_name)

        # CORRECTED evalscript - matches your working version exactly
        evalscript_optical = """
            //VERSION=3
            function setup() {
                return {
                    input: [{bands: ["B01", "B02", "B03", "B04", "B05", "B06", "B08", "B11", "B09", "CLM"]}],
                    output: {bands: 8}
                };
            }
            function evaluatePixel(sample) {
                let ndvi = index(sample.B08, sample.B04);
                let psri = (sample.B04 - sample.B02) / sample.B06;
                let msavi = (2 * sample.B08 + 1 - Math.sqrt((2 * sample.B08 + 1) ** 2 - 8 * (sample.B08 - sample.B04))) / 2;
                let sipi = (sample.B08 - sample.B02) / (sample.B08 - sample.B04);
                let gamma = 1.7;
                let gci = (sample.B08 - (sample.B03 - gamma * (sample.B02 - sample.B04))) / (sample.B08 + (sample.B03 - gamma * (sample.B02 - sample.B04)));
                let ndmi = (sample.B08 - sample.B11) / (sample.B08 + sample.B11);
                let ndre = (sample.B08 - sample.B05) / (sample.B08 + sample.B05);
                let ndwi = index(sample.B03, sample.B08);
                return [ndvi, psri, msavi, sipi, gci, ndmi, ndre, ndwi];
            }
        """

        # CORRECTED index mapping - matches your working ranges
        index_map = {
            'NDVI': [-1, 1],
            'PSRI': [-1, 1],
            'MSAVI': [-1, 1],
            'SIPI': [0, 2],      # Keep the 0-2 range as in your working version
            'GCI': [-1, 1],
            'NDMI': [-1, 1],
            'NDRE': [-1, 1],
            'NDWI': [-1, 1]
        }

        # CORRECTED index order - matches evalscript output exactly
        index_order = ['NDVI', 'PSRI', 'MSAVI', 'SIPI', 'GCI', 'NDMI', 'NDRE', 'NDWI']

        if index_name not in index_map:
            logging.error(f"Index {index_name} not found in supported indices.")
            return None

        try:
            index_idx = index_order.index(index_name)
        except ValueError as e:
            logging.error(f"Index {index_name} not found in index order: {e}")
            return None

        attempts = 0
        optical_data = None

        while attempts < max_retries and optical_data is None:
            try:
                request_optical = SentinelHubRequest(
                    evalscript=evalscript_optical,
                    input_data=[
                        SentinelHubRequest.input_data(
                            data_collection=DataCollection.SENTINEL2_L2A,
                            time_interval=time_stamp,
                            # maxcc=0.3,
                            # mosaicking_order="leastCC"
                        )
                    ],
                    responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
                    bbox=bbox,
                    size=bbox_to_dimensions(bbox, resolution=10),
                    config=config
                )

                optical_data = request_optical.get_data()
                if not optical_data or len(optical_data) == 0:
                    logging.info(f"No optical data available for {time_stamp}.")
                    return None
            except Exception as e:
                attempts += 1
                logging.warning(f"Attempt {attempts} failed: {e}. Retrying...")
                if attempts >= max_retries:
                    logging.error(f"Failed to retrieve optical data after {max_retries} attempts: {e}")
                    return None
                time.sleep(2 * attempts)

        index_data = optical_data[0][:, :, index_idx]

        # Add validation before normalization
        if np.all(np.isnan(index_data)) or np.all(index_data == 0):
            logging.warning(f"Index {index_name} contains only NaN or zero values for {date_range}")
            return None

        # CORRECTED normalization - use the working method's logic
        normalized_index_data = normalize_index(
            index_data,
            min_val=index_map[index_name][0],
            max_val=index_map[index_name][1]
        )

        bbox_epsg4326 = reproject_bounds(bbox, 'EPSG:3857', 'EPSG:4326')
        file_name = f"FIELD_{safe_farm_name}_{date_range}_{index_name}.tiff"

        result = save_data_temporarily(
            normalized_index_data,
            file_name,
            crs='EPSG:4326',
            bounds=bbox_epsg4326
        )

        result["index_name"] = index_name
        result["date_range"] = date_range

        return result

    except Exception as e:
        logging.error(f"Failed to process optical data for {index_name} in {date_range}: {e}")
        return None

def fetch_farm_data_from_api(mobile, farm_name, start_date, end_date, crop_season, area):
    """
    Fetch farm data from the API.

    Args:
        mobile (str): Mobile number
        farm_name (str): Farm name
        start_date (datetime): Start date
        end_date (datetime): End date
        crop_season (str): Crop season
        area (float): Farm area

    Returns:
        tuple: API response data or None values if API call fails
    """
    try:
        api_url = "https://3mpqory1se.execute-api.ap-south-1.amazonaws.com/prod/checkCropMonitorData_v2"
        payload = {
            "mobile": str(mobile),
            "farm_name": str(farm_name),
            "start_date": str(start_date),
            "end_date": str(end_date),
            "crop_season": str(crop_season),
            "area": area
        }

        response = requests.post(api_url, json=payload, timeout=30)
        response.raise_for_status()
        response_data = response.json()
        body_data = json.loads(response_data.get("body", "{}"))

        latest_timestamp = body_data.get("latest_timestamp")
        mean_ndvi_values = body_data.get("mean_ndvi_values")
        mean_ndmi_values = body_data.get("mean_ndmi_values")
        mean_psri_values = body_data.get("mean_psri_values")
        sowing_date = body_data.get("sowing_date")
        multi_index_stress_results = body_data.get("stress_results_data")
        # latest_timestamp = None
        # mean_ndvi_values = None
        # mean_ndmi_values = None
        # mean_psri_values = None
        # sowing_date = None
        # multi_index_stress_results = None

        return (
            latest_timestamp,
            mean_ndvi_values,
            mean_ndmi_values,
            mean_psri_values,
            sowing_date,
            multi_index_stress_results
        )
    except requests.exceptions.RequestException as e:
        logging.error(f"Error while fetching farm data: {e}")
        return None, None, None, None, None, None

def generate_date_ranges(start_date, end_date, interval_days=6):
    """
    Generate date ranges for processing.

    Args:
        start_date (datetime): Start date
        end_date (datetime): End date
        interval_days (int): Days per interval

    Returns:
        list: List of tuples with (date_range_string, time_stamp_tuple)
    """
    date_range_list = []
    current_date = end_date

    while current_date >= start_date:
        previous_date = current_date - timedelta(days=interval_days)
        # Make sure we don't go before the start date
        if previous_date < start_date:
            previous_date = start_date

        date_range = f"{previous_date.strftime('%Y-%m-%d')}_to_{current_date.strftime('%Y-%m-%d')}"
        time_stamp = (previous_date.isoformat(), current_date.isoformat())
        date_range_list.append((date_range, time_stamp))

        if previous_date == start_date:
            break

        current_date = previous_date - timedelta(days=1)

    return date_range_list

def extract_date_from_path(file_path):
    """
    Extract end date from file path.

    Args:
        file_path (str): Path to file

    Returns:
        str or None: End date string (YYYY-MM-DD) or None if not found
    """
    date_match = re.search(r"(\d{4}-\d{2}-\d{2})_to_(\d{4}-\d{2}-\d{2})", file_path)
    if date_match:
        return date_match.group(2)  # Return the end date
    return None

def process_all_data(farm_name, start_date, end_date, polygon, config, mobile, crop_season, area):
    """
    Process SAR and optical data and return TIFF file paths, data, and timestamps.

    Args:
        farm_name (str): Name of the farm.
        start_date (datetime): Start date of the data processing range.
        end_date (datetime): End date of the data processing range.
        polygon (GeoDataFrame): Geospatial polygon defining the farm area.
        config (dict): Configuration for processing.
        mobile (str): Mobile number associated with the farm.
        crop_season (str): Crop season.
        area (float): Farm area.

    Returns:
        dict: Dictionary containing TIFF file paths and associated data.
        list or None: Mean NDVI values if available, otherwise None.
        list or None: Mean NDMI values if available, otherwise None.
        list or None: Mean PSRI values if available, otherwise None.
        str or None: Latest NDVI timestamp if available, otherwise None.
        str or None: Sowing date if available, otherwise None.
        dict or None: Multi-index stress results if available, otherwise None.
    """
    try:
        # Fetch latest data from API
        api_response = fetch_farm_data_from_api(
            mobile, farm_name, start_date, end_date, crop_season, area
        )

        latest_timestamp, mean_ndvi_values, mean_ndmi_values, mean_psri_values, sowing_date, multi_index_stress_results = api_response

        # Determine the actual start date for data download
        actual_start_date = start_date
        if latest_timestamp:
            try:
                timestamp_date = datetime.fromisoformat(latest_timestamp)
                potential_start_date = timestamp_date + timedelta(days=1)

                # Check if the gap between latest timestamp and end date is less than 5 days
                days_difference = (end_date - timestamp_date).days
                if days_difference < 5:
                    logging.info(f"Gap between latest timestamp and end date is {days_difference} days (< 5 days). No new data to download.")
                    return {}, mean_ndvi_values, mean_ndmi_values, mean_psri_values, latest_timestamp, sowing_date, multi_index_stress_results

                if potential_start_date > end_date:
                    logging.info("No new data to download. Latest timestamp exceeds end date.")
                    return {}, mean_ndvi_values, mean_ndmi_values, mean_psri_values, latest_timestamp, sowing_date, multi_index_stress_results

                actual_start_date = potential_start_date
            except ValueError:
                logging.warning(f"Invalid timestamp format: {latest_timestamp}. Using original start date.")

        # Generate date ranges for processing
        date_range_list = generate_date_ranges(actual_start_date, end_date)

        # Transform polygon to Web Mercator CRS
        crs_web_mercator = CRS.from_epsg(3857)
        try:
            polygon = polygon.to_crs(crs_web_mercator.to_string())
            bounds = polygon.total_bounds
            bbox = BBox(bbox=(bounds[0], bounds[1], bounds[2], bounds[3]), crs=crs_web_mercator)
        except Exception as e:
            logging.error(f"Error transforming polygon CRS: {e}")
            return None, None, None, None, None, None, None

        all_results = {}
        ndvi_timestamps = []

        # Process SAR and optical data in parallel with rate limiting
        with ThreadPoolExecutor(max_workers=5) as executor:
            future_to_task = {}

            # # Submit SAR tasks
            # for date_range, time_stamp in date_range_list:
            #     future = executor.submit(
            #         process_sar_data,
            #         farm_name,
            #         bbox,
            #         time_stamp,
            #         date_range,
            #         config
            #     )
            #     future_to_task[future] = ('sar', date_range)

            # Submit optical tasks
            indices = ['NDVI', 'PSRI', 'MSAVI', 'SIPI', 'GCI', 'NDMI', 'NDRE', 'NDWI']
            for date_range, time_stamp in date_range_list:
                for index in indices:
                    future = executor.submit(
                        process_optical_data,
                        farm_name,
                        bbox,
                        time_stamp,
                        date_range,
                        index,
                        config
                    )
                    future_to_task[future] = ('optical', index, date_range)

            # Process results as they complete
            for future in as_completed(future_to_task):
                try:
                    result = future.result()
                    task_info = future_to_task[future]

                    if result:
                        task_type = task_info[0]

                        if task_type == 'sar':
                            date_range = task_info[1]
                            all_results[f"sar_{date_range}"] = result
                        else:  # optical
                            index_name = task_info[1]
                            date_range = task_info[2]
                            all_results[f"{index_name}_{date_range}"] = result

                            # Collect NDVI timestamps
                            if index_name == 'NDVI':
                                end_date_str = extract_date_from_path(result["path"])
                                if end_date_str:
                                    ndvi_timestamps.append(end_date_str)
                except Exception as e:
                    logging.error(f"Error processing task {future_to_task[future]}: {e}")

        # Determine which timestamp to return
        final_latest_timestamp = latest_timestamp  # Default to original API timestamp

        if ndvi_timestamps:
            # Calculate the latest NDVI date from downloaded data
            try:
                last_ndvi_date = max(ndvi_timestamps, key=lambda date: datetime.strptime(date, '%Y-%m-%d'))
                # If we downloaded new data, return the latest downloaded timestamp
                final_latest_timestamp = last_ndvi_date
                logging.info(f"New data downloaded. Updated latest timestamp to: {final_latest_timestamp}")
            except Exception as e:
                logging.error(f"Error determining latest NDVI date: {e}")
                # Keep the original timestamp if there's an error
        else:
            # If no new NDVI timestamps were collected, keep the original
            logging.info("No new NDVI data processed. Keeping original latest timestamp.")

        return all_results, mean_ndvi_values, mean_ndmi_values, mean_psri_values, final_latest_timestamp, sowing_date, multi_index_stress_results

    except Exception as e:
        logging.error(f"Error processing all data: {e}")
        return None, None, None, None, None, None, None

def mask_tiff_with_boundary(tiff_array, dataset_metadata, boundary_polygon, min_overlap_percentage=0.6):
    try:
        transform = dataset_metadata.get('transform')
        if transform is None:
            logging.warning("Transform not found in dataset metadata")
            return None

        masked_tiff = np.full(tiff_array.shape, np.nan)
        height, width = tiff_array.shape

        for row in range(height):
            for col in range(width):
                x_min, y_max = xy(transform, row, col)
                x_max, y_min = xy(transform, row + 1, col + 1)
                pixel_polygon = box(x_min, y_min, x_max, y_max)

                intersection = pixel_polygon.intersection(boundary_polygon)
                if not intersection.is_empty:
                    pixel_area = pixel_polygon.area
                    intersection_area = intersection.area
                    if (intersection_area / pixel_area) >= min_overlap_percentage:
                        masked_tiff[row, col] = tiff_array[row, col]

        return masked_tiff
    except Exception as e:
        logging.error(f"Failed to mask TIFF with farm boundary: {e}")
        return None

def extract_timestamp_indices(filename):
    match = re.search(r'\d{4}-\d{2}-\d{2}_to_\d{4}-\d{2}-\d{2}', filename)
    return match.group(0) if match else None

def process_tiff_images(tiff_images, farm_boundary_gdf=None, min_overlap_percentage=0.65):
    """
    Organize TIFF images by index, apply masking using boundary, and return means.

    Parameters:
    -----------
    tiff_images : dict
        TIFF image dictionary keyed by index+timestamp.
    farm_boundary_gdf : GeoDataFrame, optional
        GeoDataFrame containing farm boundary.
    min_overlap_percentage : float
        Minimum overlap ratio for masking pixels.

    Returns:
    --------
    masked_images : dict
        Masked TIFF arrays grouped by index.
    index_means : dict
        Mean values of masked arrays grouped by index.
    """
    organized_images = defaultdict(dict)

    # Group images by index or special key
    for key, data in tiff_images.items():
        index_match = re.match(r'([A-Z]+)_\d{4}-\d{2}-\d{2}_to_\d{4}-\d{2}-\d{2}', key)
        if index_match:
            index_name = index_match.group(1)
            organized_images[index_name][key] = data
        elif '_plant_height' in key:
            organized_images['plant_height'][key] = data
        else:
            logging.warning(f"Could not categorize file: {key}")

    if farm_boundary_gdf is None:
        return organized_images, {}

    index_keys = list(organized_images.keys())
    primary_index = "NDVI"
    if primary_index not in organized_images:
        primary_index = index_keys[0] if index_keys else None
        if not primary_index:
            logging.warning("No images available to apply masking.")
            return organized_images, {}

    # Extract boundary polygon
    if hasattr(farm_boundary_gdf, "geometry") and hasattr(farm_boundary_gdf.geometry, "iloc"):
        boundary_polygon = farm_boundary_gdf.geometry.iloc[0]
    else:
        raise ValueError("Invalid farm boundary data format.")

    # Mask and store images
    masked_images = defaultdict(dict)
    index_means = defaultdict(dict)

    for index_name, images in organized_images.items():
        for key, data in images.items():
            array = data.get("data")
            if array is None:
                continue

            dataset_metadata = {
                "transform": data.get("transform")
            }

            masked_array = mask_tiff_with_boundary(array, dataset_metadata, boundary_polygon, min_overlap_percentage)
            if masked_array is not None:
                masked_images[index_name][key] = {
                    "path": data.get("path"),
                    "data": masked_array,
                    "crs": data.get("crs"),
                    "transform": data.get("transform"),
                    "height": masked_array.shape[0],
                    "width": masked_array.shape[1]
                }

                # Calculate and store mean of non-NaN values
                mean_value = np.nanmean(masked_array)
                index_means[index_name][key] = mean_value

    return masked_images, index_means


def structure_tiff_images_by_index(tiff_images):
    """
    Organize TIFF images into a dictionary grouped by index name or plant height.
    Handles both index-based images (NDVI, NDMI, etc.) and plant height images separately.
    """
    from collections import defaultdict
    import re
    import logging

    organized_images = defaultdict(dict)

    for file_key, info in tiff_images.items():
        # Updated regex to match index names at the start of the key
        index_match = re.match(r'^(NDVI|NDMI|PSRI|MSAVI|SIPI|GCI|NDRE|NDWI)_\d{4}-\d{2}-\d{2}_to_\d{4}-\d{2}-\d{2}', file_key)
        if index_match:
            index_name = index_match.group(1)
            organized_images[index_name][file_key] = info
            continue

        # Plant height images
        if 'plant_height' in file_key:
            organized_images['plant_height'][file_key] = info
            continue

        logging.warning(f"Could not categorize file: {file_key}")

    return organized_images

####################################################### Crop Cycle Analysis ##################################################################

def prepare_data_for_analysis(mean_ndvi_values):
    """
    Prepare NDVI data:
    - Generate all possible blocks between the earliest and latest dates.
    - Detect missing blocks and assign NaN values to them.
    - Interpolate NDVI values only for missing blocks.
    """
    try:
        # Parse input into a list of (date, ndvi_value)
        parsed_dates = []
        for date_str, ndvi_value in mean_ndvi_values.items():
            try:
                date = pd.to_datetime(date_str)
                parsed_dates.append((date, ndvi_value))
            except Exception as e:
                print(f"Error parsing date {date_str}: {e}")
                continue

        if not parsed_dates:
            raise ValueError("No valid dates found after parsing")

        # Sort by date
        parsed_dates.sort(key=lambda x: x[0])

        # Calculate typical interval (use median of differences)
        intervals = []
        for i in range(1, len(parsed_dates)):
            interval = (parsed_dates[i][0] - parsed_dates[i-1][0]).days
            if interval > 0:
                intervals.append(interval)

        if not intervals:
            # Default to 7 days if we can't determine interval
            typical_interval = 7
        else:
            typical_interval = int(np.median(intervals))

        # Generate a regular time series with the computed interval
        start_date = parsed_dates[0][0]
        end_date = parsed_dates[-1][0]

        # Generate complete date range
        date_range = pd.date_range(start=start_date, end=end_date, freq=f'{typical_interval}D')

        # Create series with NaN values
        full_series = pd.Series(index=date_range, data=np.nan)

        # Fill in known values
        for date, value in parsed_dates:
            # Find the closest date in our regular series
            closest_date = date_range[np.argmin(abs(date_range - date))]
            full_series[closest_date] = value

        # Interpolate missing values
        interpolated_series = full_series.interpolate(method='linear').ffill().bfill()

        return interpolated_series, date_range

    except Exception as e:
        print(f"Error in prepare_data_for_analysis: {e}")
        raise


def detect_sowing_date(mean_ndvi_values, crop_type=None, crop_parameters=None, sowing_date=None):
    """
    Detect the sowing date based on the start of a sustained increase in smoothed NDVI.
    Enhanced to ignore early false spikes and require NDVI stability/increase after detection.

    Parameters:
        mean_ndvi_values (dict or pd.Series): NDVI values by date
        crop_type (str): Type of crop for parameter selection
        crop_parameters (dict): Dictionary of parameters for different crops
        sowing_date (str or datetime): Optional explicitly provided sowing date

    Returns:
        pd.Timestamp or str: Detected sowing date
    """
    if sowing_date is not None:
        logging.info(f"Using provided sowing date: {sowing_date}")
        return pd.to_datetime(sowing_date)

    try:
        # Extract crop-specific parameters
        if crop_type and crop_parameters and crop_type in crop_parameters:
            params = crop_parameters[crop_type]
            min_ndvi_threshold = params.get("min_ndvi_threshold", 0.1)
            rate_threshold = params.get("rate_threshold", 0.03)
            savgol_window = params.get("savgol_window", 5)
            savgol_polyorder = params.get("savgol_polyorder", 2)
        else:
            logging.warning(f"No parameters found for crop: {crop_type}. Using default parameters.")
            min_ndvi_threshold, rate_threshold = 0.15, 0.03
            savgol_window, savgol_polyorder = 7, 2

        # Convert to Series if needed
        if isinstance(mean_ndvi_values, dict):
            ndvi_series = prepare_data_for_analysis(mean_ndvi_values)
        else:
            ndvi_series = mean_ndvi_values.sort_index()

        # Adjust smoothing window if data is insufficient
        if len(ndvi_series) < savgol_window:
            logging.warning("Not enough data points for smoothing. Reducing window size.")
            savgol_window = max(3, len(ndvi_series) - 2)
            if savgol_window % 2 == 0:
                savgol_window -= 1
        savgol_polyorder = min(savgol_polyorder, savgol_window - 1)

        # Apply smoothing
        smoothed_ndvi = savgol_filter(ndvi_series.values, window_length=savgol_window, polyorder=savgol_polyorder)
        smoothed_series = pd.Series(smoothed_ndvi, index=ndvi_series.index)

        # Stability and anomaly parameters
        anomaly_window_days = 30
        drop_threshold = -0.1  # -15%
        stability_window_days = 10
        max_gap_between_points = 7  # days between NDVI observations allowed

        for i in range(len(smoothed_series) - 1):
            current_date = smoothed_series.index[i]
            next_date = smoothed_series.index[i + 1]
            current_ndvi = smoothed_series.iloc[i]
            next_ndvi = smoothed_series.iloc[i + 1]

            if current_ndvi > min_ndvi_threshold:
                rate_of_increase = (next_ndvi - current_ndvi) / max(0.001, current_ndvi)

                if rate_of_increase > rate_threshold:
                    candidate_date = current_date
                    window_end = candidate_date + pd.Timedelta(days=anomaly_window_days)
                    post_candidate = smoothed_series.loc[candidate_date:window_end]

                    if post_candidate.pct_change().min() < drop_threshold:
                        logging.info(f"Rejected candidate sowing date {candidate_date} due to NDVI drop.")
                        continue

                    # Check stability: NDVI should stay stable/increasing for at least N days
                    stable_dates = [d for d in post_candidate.index if d >= candidate_date]
                    if len(stable_dates) < 2:
                        continue

                    stable_ndvi = post_candidate.loc[stable_dates]
                    duration = (stable_ndvi.index[-1] - stable_ndvi.index[0]).days

                    # Allow minor fluctuation, but no major drop
                    if duration >= stability_window_days and all(
                        (stable_ndvi.pct_change().fillna(0) > -0.05)
                    ):
                        logging.info(f"Detected sowing date: {candidate_date}")
                        return candidate_date
                    else:
                        logging.info(f"Rejected candidate sowing date {candidate_date} due to lack of stability.")
                        continue

        # Fallback
        logging.warning("No significant and stable NDVI increase found. Using fallback date.")
        return "NA"
        # return pd.Timestamp("2024-11-20")

    except Exception as e:
        logging.error(f"Error detecting sowing date: {e}")
        raise



def determine_harvest_window(ndvi_series, sowing_date, crop_type, country, crop_harvest_parameters=None):
    """
    Determine the harvest window based on NDVI decline after reaching the peak value
    or crop-specific harvest days if decline detection fails.
    """
    try:
        if sowing_date == "NA":
            logging.warning("Sowing date is NA. Cannot determine harvest window.")
            return "NA", "NA", "NA"

        if crop_harvest_parameters is None:
            crop_harvest_parameters = {}

        if isinstance(sowing_date, str):
            sowing_date = pd.to_datetime(sowing_date)

        normalized_country = country.strip().lower()
        normalized_crop_type = crop_type.strip().replace(" ", "_").title()
        if normalized_country in crop_harvest_parameters and normalized_crop_type in crop_harvest_parameters[normalized_country]:
            params = crop_harvest_parameters[normalized_country][normalized_crop_type]
            decline_threshold = params.get("decline_threshold", 0.2)
            post_peak_buffer = params.get("post_peak_buffer", 15)
            window_length = params.get("savgol_window", 5)
            polyorder = params.get("savgol_polyorder", 2)
            min_decline_days = params.get("min_decline_days", 3)
            harvest_days = params.get("harvest_days", 100)
            min_maturity_days = params.get("min_maturity_days", 60)
            harvested_threshold = params.get("harvested_threshold", 0.15)
        else:
            logging.error(f"No parameters found for crop: {crop_type} in {country}. Using default values.")
            decline_threshold, post_peak_buffer, window_length, polyorder, min_decline_days, harvest_days, min_maturity_days, harvested_threshold = 0.2, 15, 5, 2, 3, 100, 60, 0.15

        # Filter NDVI data starting from sowing date
        ndvi_series = ndvi_series[ndvi_series.index >= sowing_date]
        if len(ndvi_series) < window_length:
            logging.warning("Insufficient NDVI data after sowing date.")
            return None, None, None

        # Apply Savitzky-Golay smoothing
        smoothed_values = savgol_filter(ndvi_series.values, window_length=window_length, polyorder=polyorder)
        smoothed_series = pd.Series(data=smoothed_values, index=ndvi_series.index).sort_index()

        # Identify peak NDVI and peak date
        peak_ndvi_date = smoothed_series.idxmax()
        peak_ndvi_value = smoothed_series.loc[peak_ndvi_date]

        # Calculate the minimum required maturity date
        min_maturity_date = sowing_date + pd.Timedelta(days=min_maturity_days)

        # If NDVI data doesn't reach the required maturity, fallback
        if smoothed_series.index[-1] < min_maturity_date:
            logging.warning("NDVI data does not cover minimum maturity days. Using fallback harvest days.")
            harvest_window_start = sowing_date + pd.Timedelta(days=harvest_days)
            harvest_window_end = harvest_window_start + pd.Timedelta(days=7)
            return harvest_window_start, harvest_window_end, None

        # Define the earliest possible decline detection date
        min_decline_date = max(
            peak_ndvi_date + pd.Timedelta(days=post_peak_buffer),
            min_maturity_date
        )
        min_decline_date = min(min_decline_date, smoothed_series.index[-1])

        post_buffer_series = smoothed_series[smoothed_series.index >= min_decline_date]
        if len(post_buffer_series) < min_decline_days:
            logging.warning("Insufficient data after post-peak buffer. Using fallback harvest days.")
            harvest_window_start = sowing_date + pd.Timedelta(days=harvest_days)
            harvest_window_end = harvest_window_start + pd.Timedelta(days=7)
            return harvest_window_start, harvest_window_end, None

        # Identify maturity date based on NDVI decline
        maturity_date = None
        for i in range(len(post_buffer_series) - min_decline_days + 1):
            window_values = post_buffer_series.iloc[i:i + min_decline_days]
            if crop_type in ["White_Onion", "Red_Onion"]:
                if all(value <= 0.55 for value in window_values) and window_values.iloc[0] >= window_values.iloc[-1]:
                    maturity_date = window_values.index[-1]
                    break
            else:
                if all(value <= peak_ndvi_value * (1 - decline_threshold) for value in window_values):
                    maturity_date = window_values.index[-1]
                    break

        # Fallback to predefined harvest days if no decline detected
        if maturity_date is None:
            logging.warning("No clear NDVI decline detected. Using fallback harvest days.")
            harvest_window_start = sowing_date + pd.Timedelta(days=harvest_days)
            harvest_window_end = harvest_window_start + pd.Timedelta(days=7)
            return harvest_window_start, harvest_window_end, None

        # Set harvest window
        harvest_window_start = maturity_date
        harvest_window_end = harvest_window_start + pd.Timedelta(days=7)

        # Determine the final harvested date
        harvested_date = None
        post_harvest_series = smoothed_series[smoothed_series.index >= harvest_window_start]
        for date, ndvi_value in post_harvest_series.items():
            if ndvi_value <= harvested_threshold:
                harvested_date = date
                break

        if harvested_date is None:
            logging.warning("No NDVI drop below harvested threshold detected. Using last day of harvest window.")
            harvested_date = harvest_window_end

        return harvest_window_start, harvest_window_end, harvested_date

    except Exception as e:
        logging.error(f"Error in harvest window determination: {e}")
        raise


def convert_index_data(index_dict, index_name='NDVI'):
    """
    Convert index_data[index_name] to {date: value} format using mid-date of the interval.
    """
    mean_values = {}
    for key, value in index_dict[index_name].items():
        try:
            # Extract mid-date from 'MSAVI_2025-05-29_to_2025-06-04'
            date_range = key.replace(f"{index_name}_", "")
            start_str, end_str = date_range.split('_to_')
            start_date = pd.to_datetime(start_str)
            end_date = pd.to_datetime(end_str)
            mid_date = start_date + (end_date - start_date) / 2
            mean_values[mid_date] = float(value)
        except Exception as e:
            print(f"Error parsing key '{key}': {e}")
    return dict(sorted(mean_values.items()))


def analyze_crop_cycle(index_data, crop_type, crop_parameters, sowing_date, crop_harvest_parameters, country, mean_ndvi=None):
    """
    Analyze the complete crop cycle including sowing date and harvest window.

    Parameters:
    -----------
    index_data : dict/list
        Primary index data (e.g., MSAVI data)
    crop_type : str
        Type of crop being analyzed
    crop_parameters : dict
        Parameters specific to the crop
    sowing_date : datetime
        Initial sowing date estimate
    crop_harvest_parameters : dict
        Parameters for harvest detection
    country : str
        Country where the crop is grown
    mean_ndvi : dict/list, optional
        Fallback NDVI values to use if index_data is empty or None

    Returns:
    --------
    tuple: (sowing_date, harvest_window_start, harvest_window_end, harvested_date)
    """
    try:
        # Check if index_data is empty or None
        if not index_data or len(index_data) == 0:
            if mean_ndvi is None or len(mean_ndvi) == 0:
                logging.error("Both index_data and mean_ndvi are empty or None.")
                return None, None, None, None

            logging.info("index_data is empty, using mean_ndvi as fallback.")
            # Pass mean_ndvi to convert_index_data function
            converted_ndvi_values = convert_index_data(mean_ndvi, index_name='NDVI')
        else:
            # Step 1: Convert MSAVI data to NDVI
            converted_ndvi_values = convert_index_data(index_data, index_name='NDVI')

        # Now prepare data for analysis with converted dates
        ndvi_series, timestamps = prepare_data_for_analysis(converted_ndvi_values)

        # Check if we need to detect sowing date or use the provided one
        if sowing_date is None or sowing_date == "":
            # Detect sowing date only if not provided
            sowing_date = detect_sowing_date(ndvi_series, crop_type=crop_type, crop_parameters=crop_parameters, sowing_date=sowing_date)

            if sowing_date is None:
                logging.error("Could not detect sowing date.")
                return None, None, None, None
        else:
            # Use the provided sowing date
            logging.info(f"Using provided sowing date: {sowing_date}")

        # Determine harvest window
        try:
            harvest_window_start, harvest_window_end, harvested_date = determine_harvest_window(
                ndvi_series, sowing_date, crop_type, country, crop_harvest_parameters
            )

            if harvest_window_start is None or harvest_window_end is None:
                logging.error("Could not detect harvest window.")
                return sowing_date, None, None, None

            return sowing_date, harvest_window_start, harvest_window_end, harvested_date

        except Exception as harvest_error:
            logging.error(f"Error in harvest window determination: {harvest_error}")
            # Return sowing date even if harvest detection fails
            return sowing_date, None, None, None

    except Exception as e:
        logging.error(f"Error in crop cycle analysis: {e}")
        # If we have a valid sowing_date at this point, preserve it
        if 'sowing_date' in locals() and sowing_date is not None:
            return sowing_date, None, None, None
        return None, None, None, None

########################################################## Stress Calculation #####################################################################

def filter_images_by_index(tiff_images, index_name):
    """Filter TIFF images by the specific index (e.g., NDVI, NDMI, PSRI) from the collected TIFF images."""
    filtered_images = tiff_images.get(index_name, {})
    print(f"Filtering images for {index_name}: Found {len(filtered_images)} images.")
    return filtered_images

def extract_first_date(date_str):
    """Extract the first date from a date string which could be either a single date or a date range."""
    match = re.match(r'^[A-Z]+_(\d{4}-\d{2}-\d{2})_to_\d{4}-\d{2}-\d{2}$', date_str)
    if match:
        return match.group(1)
    match = re.match(r'^[A-Z]+_(\d{4}-\d{2}-\d{2})$', date_str)
    if match:
        return match.group(1)
    match = re.match(r'^(\d{4}-\d{2}-\d{2})_to_\d{4}-\d{2}-\d{2}$', date_str)
    if match:
        return match.group(1)
    match = re.match(r'^(\d{4}-\d{2}-\d{2})$', date_str)
    if match:
        return match.group(1)
    print(f"WARNING: Cannot extract date from: {date_str}")
    return None

def extract_timestamp_indices(filename):
    """Extract timestamp from index image filenames."""
    match = re.search(r'(\d{4}-\d{2}-\d{2}_to_\d{4}-\d{2}-\d{2})', filename)
    if match:
        return match.group(1)
    match = re.search(r'(\d{4}-\d{2}-\d{2})', filename)
    if match:
        return match.group(1)
    return None


# Alternative function if you want to be more explicit about the expected format
def extract_first_date_flexible(date_str):
    """
    Extract the first date from various date string formats.
    Handles both prefixed and non-prefixed formats.
    """
    # Remove any index prefix (e.g., "NDVI_", "NDMI_", etc.)
    cleaned_str = re.sub(r'^[A-Z]+_', '', date_str)

    # Extract first date from range format: 2025-05-30_to_2025-06-05
    match = re.match(r'^(\d{4}-\d{2}-\d{2})(?:_to_\d{4}-\d{2}-\d{2})?$', cleaned_str)
    if match:
        return match.group(1)

    logging.warning(f"Cannot extract date from: {date_str}")
    return None

def get_growth_stage_parameters(crop_type):
    """
    Return growth stage parameters and valid transitions based on crop type.
    Includes stage windows and allowed transitions for growth stage with transition handling.

    Args:
        crop_type (str): Crop type, e.g., "Banana", "General"

    Returns:
        dict: Contains 'stages', 'stage_windows', 'valid_transitions'
    """
    crop_type_lower = crop_type.lower()
    if crop_type_lower == "banana":
        return {
            "stages": ["Planting", "Vegetative Growth", "Flowering", "Fruit Development", "Maturity"],
            "stage_windows": {
                "Planting": (0.00, 0.10),
                "Vegetative Growth": (0.10, 0.50),
                "Flowering": (0.50, 0.70),
                "Fruit Development": (0.70, 0.90),
                "Maturity": (0.90, 1.01),
            },
            "valid_transitions": {
                "Planting": ["Planting", "Vegetative Growth"],
                "Vegetative Growth": ["Vegetative Growth", "Flowering"],
                "Flowering": ["Flowering", "Fruit Development"],
                "Fruit Development": ["Fruit Development", "Maturity"],
                "Maturity": ["Maturity"],
            }
        }
    else:
        # General crop defaults
        return {
            "stages": ["Emergence", "Vegetative Growth", "Reproductive (Flowering/Fruit Set)",
                       "Fruit Development", "Maturity / Senescence"],
            "stage_windows": {
                "Emergence": (0.00, 0.10),
                "Vegetative Growth": (0.10, 0.50),
                "Reproductive (Flowering/Fruit Set)": (0.45, 0.75),
                "Fruit Development": (0.75, 0.90),
                "Maturity / Senescence": (0.90, 1.01),
            },
            "valid_transitions": {
                "Emergence": ["Emergence", "Vegetative Growth"],
                "Vegetative Growth": ["Vegetative Growth", "Reproductive (Flowering/Fruit Set)"],
                "Reproductive (Flowering/Fruit Set)": ["Reproductive (Flowering/Fruit Set)", "Fruit Development"],
                "Fruit Development": ["Fruit Development", "Maturity / Senescence"],
                "Maturity / Senescence": ["Maturity / Senescence"],
            }
        }

def classify_growth_stage_with_transition(
    timestamp, sowing_date, harvest_date,
    vegetation_indices, previous_stage=None,
    transition_threshold=0.2, crop_type="General"
):
    """
    Classify growth stage with transition state handling.
    Combines time-progress and VI conditions with stage transitions and confidence scores.

    Args:
        timestamp (datetime-like): current datetime
        sowing_date (datetime-like): sowing/start date
        harvest_date (datetime-like): harvest/end date
        vegetation_indices (dict): dict of index_name -> float (single values of indices)
        previous_stage (str or None): previously detected growth stage
        transition_threshold (float): threshold difference to detect transition (between primary and secondary)
        crop_type (str): crop type

    Returns:
        tuple:
          primary_stage (str)
          transition_info (dict): {
              'primary': str,
              'secondary': str or None,
              'is_transition': bool,
              'confidence_scores': dict of stage -> float,
              'raw_scores': dict of stage -> float
          }
    """
    # Convert to datetime if needed
    timestamp = pd.to_datetime(timestamp)
    sowing_date = pd.to_datetime(sowing_date)
    harvest_date = pd.to_datetime(harvest_date)

    params = get_growth_stage_parameters(crop_type)
    stages = params["stages"]
    stage_windows = params["stage_windows"]
    valid_transitions = params["valid_transitions"]

    # Handle edge cases
    total_days = (harvest_date - sowing_date).days
    if total_days <= 0:
        return "Unknown", {
            "primary": "Unknown", "secondary": None,
            "is_transition": False,
            "confidence_scores": {"Unknown": 1.0}, "raw_scores": {}
        }
    days_elapsed = (timestamp - sowing_date).days
    if days_elapsed < 0:
        return "Pre-Sowing", {
            "primary": "Pre-Sowing", "secondary": None,
            "is_transition": False,
            "confidence_scores": {"Pre-Sowing": 1.0}, "raw_scores": {}
        }
    if timestamp > harvest_date:
        return "Post-Harvest", {
            "primary": "Post-Harvest", "secondary": None,
            "is_transition": False,
            "confidence_scores": {"Post-Harvest": 1.0}, "raw_scores": {}
        }

    progress = days_elapsed / total_days

    # VI conditions for scoring per stage
    vi_conditions = get_vegetative_index_conditions(crop_type)

    # Time based scoring - 1 if inside stage window, otherwise Gaussian fall off (smooth)
    time_scores = {}
    for stage in stages:
        start_p, end_p = stage_windows[stage]
        center = (start_p + end_p) / 2
        if start_p <= progress <= end_p:
            time_scores[stage] = 1.0
        else:
            time_scores[stage] = np.exp(-((progress - center) ** 2) / 0.01)

    # VI based scoring for current index values and stage-specific VI conditions
    vi_scores = {stage: 0.0 for stage in stages}
    for stage in stages:
        if stage not in vi_conditions:
            continue
        conds = vi_conditions[stage]
        for index, val in vegetation_indices.items():
            if val is None or (isinstance(val, float) and np.isnan(val)):
                continue
            # Process conditions if present
            if f"{index}_low" in conds and val >= conds[f"{index}_low"]:
                if f"{index}_high" in conds and val < conds[f"{index}_high"]:
                    vi_scores[stage] += 1.0
                elif f"{index}_high" not in conds:
                    vi_scores[stage] += 1.0
            elif f"{index}_min" in conds and val >= conds[f"{index}_min"]:
                vi_scores[stage] += 1.0
            elif f"{index}_max" in conds and val < conds[f"{index}_max"]:
                vi_scores[stage] += 0.5
            elif f"{index}_min" not in conds and f"{index}_max" not in conds:
                # fallback if no min/max but value present, just increment a tiny amount to indicate presence
                vi_scores[stage] += 0.1

    # Combine VI and time scores (weights: VI 60%, time 40%)
    combined_scores = {}
    for stage in stages:
        combined_scores[stage] = 0.6 * vi_scores.get(stage, 0) + 0.4 * time_scores.get(stage, 0)

    # Respect valid transitions if previous stage known
    allowed_stages = valid_transitions.get(previous_stage, stages) if previous_stage else stages
    filtered_scores = {stage: combined_scores.get(stage, 0) for stage in allowed_stages}

    total_score = sum(filtered_scores.values())
    if total_score == 0:
        return "Unknown", {
            "primary": "Unknown", "secondary": None,
            "is_transition": False,
            "confidence_scores": {"Unknown": 1.0}, "raw_scores": combined_scores
        }

    normalized_scores = {stage: score / total_score for stage, score in filtered_scores.items()}
    sorted_stages = sorted(normalized_scores.items(), key=lambda x: x[1], reverse=True)

    primary, primary_score = sorted_stages[0]
    secondary, secondary_score = (sorted_stages[1] if len(sorted_stages) > 1 else (None, 0.0))
    is_transition = (secondary is not None) and ((primary_score - secondary_score) <= transition_threshold)

    transition_info = {
        "primary": primary,
        "secondary": secondary,
        "is_transition": is_transition,
        "confidence_scores": normalized_scores,
        "raw_scores": combined_scores
    }
    return primary, transition_info

def get_vegetative_index_conditions(crop_type):
    """
    Vegetation index conditions for growth stage classification.
    Defines thresholds for indices for each growth stage and crop type.

    Args:
        crop_type (str): Crop type

    Returns:
        dict: {stage: {condition_name: threshold_value}}
    """
    crop_type_lower = crop_type.lower()
    # Base conditions
    base_conditions = {
        "Emergence": {
            "NDVI_low": 0.0, "NDVI_high": 0.3,
            "NDMI_low": 0.0, "NDMI_high": 0.3,
            "MSAVI_low": 0.0, "MSAVI_high": 0.4
        },
        "Vegetative Growth": {
            "NDVI_low": 0.3, "NDVI_high": 0.6,
            "GCI_min": 0.4,
            "NDRE_min": 0.3
        },
        "Reproductive (Flowering/Fruit Set)": {
            "NDVI_low": 0.6, "NDVI_high": 0.8,
            "NDRE_min": 0.4
        },
        "Fruit Development": {
            "PSRI_min": 0.3,
            "NDVI_min": 0.6
        },
        "Maturity / Senescence": {
            "PSRI_min": 0.45,
            "NDVI_max": 0.5,
            "NDMI_max": 0.25
        }
    }
    if crop_type_lower == "banana":
        banana_conditions = {
            "Planting": {
                "NDVI_low": 0.0, "NDVI_high": 0.3,
                "NDMI_low": 0.0, "NDMI_high": 0.3
            },
            "Vegetative Growth": {
                "NDVI_low": 0.3, "NDVI_high": 0.6,
                "GCI_min": 0.45
            },
            "Flowering": {
                "NDVI_low": 0.6, "NDVI_high": 0.8,
                "NDRE_min": 0.45
            },
            "Fruit Development": {
                "PSRI_min": 0.3,
                "NDVI_min": 0.65
            },
            "Maturity": {
                "PSRI_min": 0.45,
                "NDVI_max": 0.55
            }
        }
        return banana_conditions
    return base_conditions

def weighted_average(index_name, param_dict, confidence_scores):
    """
    Compute weighted average for an index across growth stages given confidence scores.

    Args:
        index_name (str): Vegetation index name
        param_dict (dict): dict of stage -> {index_name: value}
        confidence_scores (dict): dict of stage -> weight

    Returns:
        float: weighted average, or None if no valid data
    """
    total = 0.0
    weight_sum = 0.0
    for stage, confidence in confidence_scores.items():
        stage_dict = param_dict.get(stage, {})
        val = stage_dict.get(index_name)
        if val is not None and confidence > 0:
            total += confidence * val
            weight_sum += confidence
    if weight_sum > 0:
        return total / weight_sum
    else:
        return None

def calculate_real_time_stress_percentage(index_data, transition_info, crop_type="general"):
    """
    Improved stress percentage calculation supporting multiple indices with
    weighting by growth stage confidence and crop specific parameters.

    Args:
        index_data (dict): {index_name: np.array of pixel values}
        transition_info (dict): dict containing 'confidence_scores', 'primary', etc.
        crop_type (str): Crop type

    Returns:
        (float, dict): final stress percentage, detailed results per index
    """
    # Default reference values and weights per crop and growth stage, based on method 1 data

    crop = crop_type.lower()
    # Use base reference values for general crops
    reference_values = {
        "Emergence": {"NDVI": 0.25, "MSAVI": 0.35, "NDMI": 0.20, "NDRE": 0.15, "GCI": 0.40, "PSRI": 0.60},
        "Vegetative Growth": {"NDVI": 0.55, "MSAVI": 0.50, "NDMI": 0.40, "NDRE": 0.35, "GCI": 0.60, "PSRI": 0.20},
        "Reproductive (Flowering/Fruit Set)": {"NDVI": 0.70, "MSAVI": 0.60, "NDMI": 0.50, "NDRE": 0.45, "GCI": 0.80, "PSRI": 0.20},
        "Fruit Development": {"NDVI": 0.65, "MSAVI": 0.50, "NDMI": 0.45, "NDRE": 0.40, "GCI": 0.70, "PSRI": 0.30},
        "Maturity / Senescence": {"NDVI": 0.40, "MSAVI": 0.45, "NDMI": 0.30, "NDRE": 0.30, "GCI": 0.40, "PSRI": 0.65},
        "Unknown": {"NDVI": 0.50, "MSAVI": 0.55, "NDMI": 0.35, "NDRE": 0.30, "GCI": 0.60, "PSRI": 0.20},
    }
    stage_weights = {
        "Emergence": {"NDVI": 0.20, "MSAVI": 0.40, "NDMI": 0.15, "NDRE": 0.05, "GCI": 0.15, "PSRI": 0.00},
        "Vegetative Growth": {"NDVI": 0.25, "MSAVI": 0.10, "NDMI": 0.15, "NDRE": 0.20, "GCI": 0.25, "PSRI": 0.00},
        "Reproductive (Flowering/Fruit Set)": {"NDVI": 0.20, "MSAVI": 0.05, "NDMI": 0.20, "NDRE": 0.25, "GCI": 0.15, "PSRI": 0.10},
        "Fruit Development": {"NDVI": 0.15, "MSAVI": 0.05, "NDMI": 0.20, "NDRE": 0.20, "GCI": 0.10, "PSRI": 0.20},
        "Maturity / Senescence": {"NDVI": 0.10, "MSAVI": 0.05, "NDMI": 0.15, "NDRE": 0.15, "GCI": 0.05, "PSRI": 0.30},
        "Unknown": {"NDVI": 0.25, "MSAVI": 0.10, "NDMI": 0.20, "NDRE": 0.15, "GCI": 0.15, "PSRI": 0.10},
    }
    tolerance_thresholds = {
        "Emergence": {"NDVI": 0.25, "MSAVI": 0.25, "NDMI": 0.30, "NDRE": 0.35, "GCI": 0.25, "PSRI": 0.40},
        "Vegetative Growth": {"NDVI": 0.20, "MSAVI": 0.20, "NDMI": 0.25, "NDRE": 0.25, "GCI": 0.20, "PSRI": 0.30},
        "Reproductive (Flowering/Fruit Set)": {"NDVI": 0.15, "MSAVI": 0.15, "NDMI": 0.20, "NDRE": 0.20, "GCI": 0.15, "PSRI": 0.25},
        "Fruit Development": {"NDVI": 0.15, "MSAVI": 0.15, "NDMI": 0.20, "NDRE": 0.20, "GCI": 0.15, "PSRI": 0.20},
        "Maturity / Senescence": {"NDVI": 0.20, "MSAVI": 0.20, "NDMI": 0.25, "NDRE": 0.25, "GCI": 0.25, "PSRI": 0.20},
        "Unknown": {"NDVI": 0.20, "MSAVI": 0.20, "NDMI": 0.25, "NDRE": 0.25, "GCI": 0.20, "PSRI": 0.25},
    }
    # Override for banana crop
    if crop == "banana":
        reference_values = {
            "Emergence": {"NDVI": 0.30, "MSAVI": 0.40, "NDMI": 0.25, "NDRE": 0.20, "GCI": 0.45, "PSRI": 0.50},
            "Vegetative Growth": {"NDVI": 0.60, "MSAVI": 0.55, "NDMI": 0.45, "NDRE": 0.40, "GCI": 0.65, "PSRI": 0.25},
            "Reproductive (Flowering/Fruit Set)": {"NDVI": 0.75, "MSAVI": 0.65, "NDMI": 0.55, "NDRE": 0.50, "GCI": 0.85, "PSRI": 0.15},
            "Fruit Development": {"NDVI": 0.70, "MSAVI": 0.60, "NDMI": 0.50, "NDRE": 0.45, "GCI": 0.80, "PSRI": 0.25},
            "Maturity / Senescence": {"NDVI": 0.45, "MSAVI": 0.50, "NDMI": 0.35, "NDRE": 0.35, "GCI": 0.50, "PSRI": 0.55},
            "Unknown": {"NDVI": 0.50, "MSAVI": 0.55, "NDMI": 0.35, "NDRE": 0.30, "GCI": 0.60, "PSRI": 0.20},
        }
        stage_weights = {
            "Emergence": {"NDVI": 0.15, "MSAVI": 0.35, "NDMI": 0.15, "NDRE": 0.10, "GCI": 0.20, "PSRI": 0.05},
            "Vegetative Growth": {"NDVI": 0.30, "MSAVI": 0.10, "NDMI": 0.20, "NDRE": 0.20, "GCI": 0.15, "PSRI": 0.05},
            "Reproductive (Flowering/Fruit Set)": {"NDVI": 0.25, "MSAVI": 0.05, "NDMI": 0.20, "NDRE": 0.25, "GCI": 0.15, "PSRI": 0.10},
            "Fruit Development": {"NDVI": 0.20, "MSAVI": 0.05, "NDMI": 0.25, "NDRE": 0.20, "GCI": 0.10, "PSRI": 0.20},
            "Maturity / Senescence": {"NDVI": 0.10, "MSAVI": 0.05, "NDMI": 0.15, "NDRE": 0.15, "GCI": 0.05, "PSRI": 0.30},
            "Unknown": {"NDVI": 0.25, "MSAVI": 0.10, "NDMI": 0.20, "NDRE": 0.15, "GCI": 0.15, "PSRI": 0.10},
        }
        tolerance_thresholds = {
            "Emergence": {"NDVI": 0.20, "MSAVI": 0.25, "NDMI": 0.25, "NDRE": 0.30, "GCI": 0.20, "PSRI": 0.35},
            "Vegetative Growth": {"NDVI": 0.15, "MSAVI": 0.20, "NDMI": 0.20, "NDRE": 0.20, "GCI": 0.15, "PSRI": 0.25},
            "Reproductive (Flowering/Fruit Set)": {"NDVI": 0.15, "MSAVI": 0.15, "NDMI": 0.20, "NDRE": 0.20, "GCI": 0.15, "PSRI": 0.20},
            "Fruit Development": {"NDVI": 0.15, "MSAVI": 0.15, "NDMI": 0.20, "NDRE": 0.20, "GCI": 0.15, "PSRI": 0.20},
            "Maturity / Senescence": {"NDVI": 0.20, "MSAVI": 0.20, "NDMI": 0.25, "NDRE": 0.25, "GCI": 0.25, "PSRI": 0.20},
            "Unknown": {"NDVI": 0.20, "MSAVI": 0.20, "NDMI": 0.25, "NDRE": 0.25, "GCI": 0.20, "PSRI": 0.25},
        }

    index_results = {}
    total_weight = 0.0
    total_weighted_stress = 0.0

    for index_name, index_array in index_data.items():
        # Skip if data missing or all nan
        if index_array is None or not isinstance(index_array, np.ndarray):
            continue

        valid_mask = ~np.isnan(index_array)
        if index_name in ["NDVI", "MSAVI", "NDRE"]:
            valid_mask &= (index_array > 0.1)
        valid_pixels = index_array[valid_mask]

        if valid_pixels.size < 10:
            continue

        expected_val = weighted_average(index_name, reference_values, transition_info["confidence_scores"])
        tolerance = weighted_average(index_name, tolerance_thresholds, transition_info["confidence_scores"])
        weight = weighted_average(index_name, stage_weights, transition_info["confidence_scores"])
        if expected_val is None or weight == 0 or tolerance is None:
            continue

        actual_mean = np.nanmean(valid_pixels)

        if index_name in ["NDVI", "MSAVI", "NDMI", "NDRE", "GCI"]:
            lower_bound = expected_val * (1 - tolerance)
            stress_pixels = valid_pixels < lower_bound
            relative_deviation = max(0, (expected_val - actual_mean) / expected_val)
        else:
            upper_bound = expected_val * (1 + tolerance)
            stress_pixels = valid_pixels > upper_bound
            relative_deviation = max(0, (actual_mean - expected_val) / expected_val)

        # Apply minimum deviation threshold
        if relative_deviation < 0.05:
            relative_deviation = 0

        stress_count = np.sum(stress_pixels)
        stress_percentage = (stress_count / valid_pixels.size) * 100
        normalized_stress = min(100, (relative_deviation / (2 * tolerance)) * 100)
        combined_stress = 0.3 * stress_percentage + 0.7 * normalized_stress

        # Dampening stress value if transition state (max 50% dampening)
        if transition_info.get("is_transition", False):
            max_conf = max(transition_info["confidence_scores"].values(), default=1.0)
            dampening = 1 - max_conf
            combined_stress *= (1 - 0.5 * dampening)

        index_results[index_name] = {
            "actual_mean": float(actual_mean),
            "expected_value": expected_val,
            "tolerance": tolerance,
            "deviation_pct": float(relative_deviation * 100),
            "stress_percentage": float(stress_percentage),
            "normalized_stress": float(normalized_stress),
            "combined_stress": float(combined_stress),
            "weight": weight,
            "contribution": float(combined_stress * weight),
        }
        total_weight += weight
        total_weighted_stress += combined_stress * weight

    final_stress = total_weighted_stress / total_weight if total_weight > 0 else 0.0

    results = {
        "overall_stress_percentage": final_stress,
        "indices": index_results,
        "growth_stage": transition_info.get("primary", "Unknown"),
        "growth_stage_secondary": transition_info.get("secondary"),
        "is_transition": transition_info.get("is_transition", False),
        "total_weight_applied": total_weight,
        "confidence_scores": transition_info.get("confidence_scores", {}),
    }

    return final_stress, results

def calc_corr(a, b):
    """
    Calculate Pearson correlation coefficient between two arrays.
    Returns None if insufficient data or invalid input.

    Args:
        a, b (np.array): arrays with values (possibly containing NaNs)

    Returns:
        float or None: correlation coefficient or None if cannot be computed.
    """
    try:
        if a is None or b is None:
            return None
        mask = ~np.isnan(a) & ~np.isnan(b)
        if np.sum(mask) < 10:
            return None
        a_masked = a[mask]
        b_masked = b[mask]
        if np.std(a_masked) == 0 or np.std(b_masked) == 0:
            return None
        return np.corrcoef(a_masked, b_masked)[0, 1]
    except Exception as e:
        logging.warning(f"Correlation calculation error: {e}")
        return None


def classify_stress_type(
    index_data, transition_info, threshold=0.7, crop_type="General"
):
    """
    Classify stress type based on multi-index correlations, stress percentage,
    and crop-specific thresholds combined and improved.

    Args:
        index_data (dict): index_name -> np.array of pixel values
        transition_info (dict): includes growth stage and confidence scores
        threshold (float): stress percentage threshold to consider healthy vs stressed
        crop_type (str): crop type

    Returns:
        str: Classified stress type
    """
    try:
        # Calculate stress percentage with detailed index contributions
        stress_percentage, stress_details = calculate_real_time_stress_percentage(
            index_data, transition_info, crop_type
        )

        # ✅ NEW CONDITION: If stress percentage is less than 20%, return as Healthy
        # This condition is correctly placed BEFORE the threshold check
        if stress_percentage < 20:
            return "Healthy"

        # Early healthy cutoff (this becomes redundant if threshold >= 20)
        if stress_percentage < threshold:
            return "Healthy"

        # Extract indices arrays
        ndvi = index_data.get("NDVI")
        ndmi = index_data.get("NDMI")
        psri = index_data.get("PSRI")
        msavi = index_data.get("MSAVI")
        gci = index_data.get("GCI")
        ndre = index_data.get("NDRE")
        sipi = index_data.get("SIPI")

        # Calculate correlations
        correlations = {
            "ndvi_ndmi": calc_corr(ndvi, ndmi),
            "ndvi_msavi": calc_corr(ndvi, msavi),
            "ndvi_gci": calc_corr(ndvi, gci),
            "ndvi_psri": calc_corr(ndvi, psri),
            "psri_ndre": calc_corr(psri, ndre),
            "ndvi_ndre": calc_corr(ndvi, ndre),
            "ndvi_sipi": calc_corr(ndvi, sipi),
        }

        # Calculate means
        means = {k: np.nanmean(index_data[k]) if index_data.get(k) is not None else None for k in [
            "NDVI", "NDMI", "PSRI", "GCI", "NDRE", "MSAVI", "SIPI"
        ]}

        growth_stage = transition_info.get("primary", "Unknown")
        crop = crop_type.lower()

        # Thresholds for classification adapted from both methods per crop and stage
        thresholds = get_stress_thresholds(crop_type, growth_stage)

        # Contribution by index
        contributions = {
            idx: data.get("contribution", 0)
            for idx, data in stress_details.get("indices", {}).items()
        }
        # Get max contributing index
        max_index = max(contributions.items(), key=lambda x: x[1])[0] if contributions else "NDVI"

        # Detailed logic for stress type considering crop type and growth stage
        # Categorize stress with enhanced correlation and mean index checks

        # Poor Germination or Emergence issues
        if growth_stage in ["Emergence", "Planting"]:
            corr_thresh = thresholds.get("correlation", {}).get("ndvi_msavi_low", 0.3)
            if correlations["ndvi_msavi"] is not None and correlations["ndvi_msavi"] < corr_thresh:
                return "Poor Germination"
            msavi_thresh = thresholds.get("mean", {}).get("msavi_low", 0.2)
            if means["MSAVI"] is not None and means["MSAVI"] < msavi_thresh:
                return "Poor Germination"
            ndmi_thresh = thresholds.get("mean", {}).get("ndmi_low", 0.2)
            if max_index == "NDMI" or (means["NDMI"] is not None and means["NDMI"] < ndmi_thresh):
                return "Water Stress"
            return "General Vegetation Stress"

        # Vegetative Growth issues
        if growth_stage == "Vegetative Growth":
            gci_corr_thresh = thresholds.get("correlation", {}).get("ndvi_gci_low", 0.3)
            ndmi_corr_thresh = thresholds.get("correlation", {}).get("ndvi_ndmi_low", 0.3)
            psri_corr_thresh = thresholds.get("correlation", {}).get("ndvi_psri_high", 0.4)
            ndre_corr_thresh = thresholds.get("correlation", {}).get("ndvi_ndre_low", -0.3)

            if max_index in ["GCI", "NDRE"]:
                return "Nutrient Deficiency"
            if (correlations["ndvi_gci"] is not None) and (correlations["ndvi_gci"] < gci_corr_thresh):
                return "Nutrient Deficiency"
            if max_index == "NDMI" or (correlations["ndvi_ndmi"] is not None and correlations["ndvi_ndmi"] < ndmi_corr_thresh):
                return "Water Stress"
            if max_index == "PSRI" or (
                correlations["ndvi_psri"] is not None and correlations["ndvi_psri"] > psri_corr_thresh and
                correlations["ndvi_ndre"] is not None and correlations["ndvi_ndre"] < ndre_corr_thresh):
                return "Disease/Pest Stress"
            return "General Vegetation Stress"

        # Reproductive / Flowering issues
        if growth_stage in ["Reproductive (Flowering/Fruit Set)", "Flowering"]:
            psri_mean_thresh = thresholds.get("mean", {}).get("psri_high", 0.4)
            psri_corr_thresh = thresholds.get("correlation", {}).get("ndvi_psri_high", 0.3)
            ndmi_mean_thresh = thresholds.get("mean", {}).get("ndmi_low", 0.3)
            ndmi_corr_thresh = thresholds.get("correlation", {}).get("ndvi_ndmi_low", 0.3)
            gci_mean_thresh = thresholds.get("mean", {}).get("gci_low", 0.3)
            gci_corr_thresh = thresholds.get("correlation", {}).get("ndvi_gci_low", 0.3)

            if max_index == "PSRI" or (means["PSRI"] is not None and means["PSRI"] > psri_mean_thresh and
                                     correlations["ndvi_psri"] is not None and correlations["ndvi_psri"] > psri_corr_thresh):
                return "Disease/Pest Stress"
            if max_index == "NDMI" or (means["NDMI"] is not None and means["NDMI"] < ndmi_mean_thresh and
                                     correlations["ndvi_ndmi"] is not None and correlations["ndvi_ndmi"] < ndmi_corr_thresh):
                return "Water Stress"
            if max_index in ["GCI", "NDRE"] or (means["GCI"] is not None and means["GCI"] < gci_mean_thresh and
                                               correlations["ndvi_gci"] is not None and correlations["ndvi_gci"] < gci_corr_thresh):
                return "Nutrient Deficiency"
            return "General Vegetation Stress"

        # Fruit Development issues
        if growth_stage == "Fruit Development":
            psri_mean_thresh = thresholds.get("mean", {}).get("psri_high", 0.5)
            ndmi_mean_thresh = thresholds.get("mean", {}).get("ndmi_low", 0.3)
            gci_mean_thresh = thresholds.get("mean", {}).get("gci_low", 0.3)

            if max_index == "PSRI" or (means["PSRI"] is not None and means["PSRI"] > psri_mean_thresh):
                return "Disease/Pest Stress"
            if max_index == "NDMI" or (means["NDMI"] is not None and means["NDMI"] < ndmi_mean_thresh):
                return "Water Stress"
            if max_index in ["GCI", "NDRE"] or (means["GCI"] is not None and means["GCI"] < gci_mean_thresh):
                return "Nutrient Deficiency"
            return "General Vegetation Stress"

        # Maturity / Senescence issues
        if growth_stage in ["Maturity / Senescence", "Maturity"]:
            ndmi_low_thresh = thresholds.get("mean", {}).get("ndmi_low", 0.2)
            ndmi_high_thresh = thresholds.get("mean", {}).get("ndmi_high", 0.6)
            psri_high_thresh = thresholds.get("mean", {}).get("psri_high", 0.7)

            if max_index == "NDMI":
                if means["NDMI"] is not None and means["NDMI"] < ndmi_low_thresh:
                    return "Water Stress"
                if means["NDMI"] is not None and means["NDMI"] > ndmi_high_thresh:
                    return "Over-Irrigation"
            if max_index == "PSRI" or (means["PSRI"] is not None and means["PSRI"] > psri_high_thresh):
                return "Disease/Pest Stress"
            return "General Vegetation Stress"

        # Fallback default
        return "General Vegetation Stress"
    except Exception as ex:
        logging.error(f"Error during stress classification: {ex}")
        return "Unknown"

def get_stress_thresholds(crop_type, growth_stage):
    """
    Get stress classification thresholds based on crop type and growth stage.
    Combines base method thresholds with banana-specific overrides.

    Args:
        crop_type (str)
        growth_stage (str)

    Returns:
        dict of thresholds
    """
    base_thresholds = {
        "Emergence": {
            "correlation": {
                "ndvi_msavi_low": 0.3,
                "ndvi_ndmi_low": 0.3
            },
            "mean": {
                "msavi_low": 0.2,
                "ndmi_low": 0.2
            }
        },
        "Vegetative Growth": {
            "correlation": {
                "ndvi_gci_low": 0.3,
                "ndvi_ndmi_low": 0.3,
                "ndvi_psri_high": 0.4,
                "ndvi_ndre_low": -0.3
            },
            "mean": {
                "ndmi_low": 0.3
            }
        },
        "Reproductive (Flowering/Fruit Set)": {
            "correlation": {
                "ndvi_psri_high": 0.3,
                "ndvi_ndmi_low": 0.3,
                "ndvi_gci_low": 0.3
            },
            "mean": {
                "psri_high": 0.4,
                "ndmi_low": 0.3,
                "gci_low": 0.3
            }
        },
        "Fruit Development": {
            "mean": {
                "psri_high": 0.5,
                "ndmi_low": 0.3,
                "gci_low": 0.3
            }
        },
        "Maturity / Senescence": {
            "mean": {
                "ndmi_low": 0.2,
                "ndmi_high": 0.6,
                "psri_high": 0.7
            }
        }
    }
    banana_thresholds = {
        "Planting": {
            "correlation": {"ndvi_msavi_low": 0.3}
        },
        "Vegetative Growth": {
            "correlation": {
                "ndvi_gci_low": 0.3,
                "ndvi_ndmi_low": 0.3
            },
            "mean": {
                "ndmi_low": 0.3
            }
        },
        "Flowering": {
            "mean": {
                "psri_high": 0.4,
                "ndmi_low": 0.3
            }
        },
        "Fruit Development": {
            "mean": {
                "psri_high": 0.5,
                "ndmi_low": 0.3
            }
        },
        "Maturity": {
            "mean": {
                "ndmi_low": 0.2,
                "psri_high": 0.7
            }
        }
    }
    crop = crop_type.lower()
    if crop == "banana":
        return banana_thresholds.get(growth_stage, {})
    else:
        return base_thresholds.get(growth_stage, {})

def process_crop_monitoring_with_cycle_analysis(
    tiff_images,
    farm_boundary_gdf,
    crop_type,
    crop_parameters,
    crop_harvest_parameters,
    country,
    sowing_date,
    harvest_start,
    harvest_end,
    harvested_date,
    index_data,
    multi_index_stress_results=None,
    deviant_threshold=0.7,
):
    """
    Integrated function for crop monitoring including crop cycle analysis.
    Includes detailed logging to diagnose NDVI None/NaN issues.
    """
    try:
        multi_index_stress_results = multi_index_stress_results or []
        index_keys = ["NDVI", "NDMI", "PSRI", "MSAVI", "NDRE", "GCI", "SIPI"]

        # Extract and standardize images by timestamp
        standardized_images = {key: {} for key in index_keys}
        for key in index_keys:
            dataset = filter_images_by_index(tiff_images, key)
            for f, data in dataset.items():
                timestamp = extract_timestamp_indices(f)
                if timestamp:
                    standardized_images[key][timestamp] = (data.get("data"), data)
            print(f"Extracted {len(standardized_images[key])} images for index {key}")

        # Log standardized images structure
        print("Standardized images structure:")
        for key, value in standardized_images.items():
            print(f"  {key}: {len(value)} entries")

        # Log NDVI keys and sample data to debug missing NDVI
        ndvi_timestamps = list(standardized_images["NDVI"].keys())
        print(f"NDVI timestamps extracted: {ndvi_timestamps[:10]} ... total {len(ndvi_timestamps)}")
        for ts in ndvi_timestamps[:5]:
            ndvi_data_sample = standardized_images["NDVI"][ts][0]
            if ndvi_data_sample is None:
                print(f"WARNING: NDVI data is None for timestamp {ts} after extraction.")
            else:
                shape = getattr(ndvi_data_sample, 'shape', 'N/A')
                print(f"NDVI data present for timestamp {ts} (type: {type(ndvi_data_sample)}; shape: {shape})")

        # Check if NDVI data is available at all
        if not standardized_images["NDVI"]:
            print("WARNING: No NDVI data found after extraction. Returning previous results.")
            return multi_index_stress_results, {}, (sowing_date, harvest_start, harvest_end, harvested_date)

        # Convert dates to pd.Timestamp with fallbacks
        sowing_date_ts = pd.Timestamp(sowing_date) if sowing_date is not None else pd.Timestamp.min
        if harvested_date is not None:
            harvest_end_ts = pd.Timestamp(harvested_date)
        elif harvest_end is not None:
            harvest_end_ts = pd.Timestamp(harvest_end)
        else:
            harvest_end_ts = pd.Timestamp.max

        sowing_date_dt = sowing_date_ts.to_pydatetime()
        harvest_end_dt = harvest_end_ts.to_pydatetime()

        # Filter timestamps from index_data['NDVI'] within sowing and harvest window
        filtered_timestamps = {}
        for ts in index_data.get("NDVI", {}):
            date_str = extract_first_date(ts)
            if date_str is None:
                continue
            ts_date = pd.to_datetime(date_str).to_pydatetime()
            if sowing_date_dt <= ts_date <= harvest_end_dt:
                filtered_timestamps[ts] = ts_date

        print(f"Filtered timestamps between sowing and harvest: {list(filtered_timestamps.keys())[:10]} ... total {len(filtered_timestamps)}")

        # Growth Stage Classification with transition logic
        growth_stages = {}
        transition_infos = {}
        previous_stage = None

        sorted_timestamps = sorted(filtered_timestamps.items(), key=lambda x: x[1])
        for ts, ts_date in sorted_timestamps:
            # FIXED: Extract the base timestamp (without index prefix) for consistent lookup
            base_timestamp = extract_timestamp_indices(ts)  # This extracts just the date part

            # Extract aggregated vegetation indices means from index_data dictionaries
            # Create the correct key format for each index
            vegetation_indices = {}
            for key in index_keys:
                # Create the full key for this index (e.g., 'NDMI_2025-05-31_to_2025-06-06')
                full_key = f"{key}_{base_timestamp}"
                vegetation_indices[key] = index_data.get(key, {}).get(full_key, np.nan)

            print(f"Timestamp {ts}: Vegetation Indices: {vegetation_indices}")

            # Classify growth stage and transitions
            current_stage, transition_info = classify_growth_stage_with_transition(
                timestamp=ts_date,
                sowing_date=sowing_date_dt,
                harvest_date=harvest_end_dt,
                vegetation_indices=vegetation_indices,
                previous_stage=previous_stage,
                crop_type=crop_type
            )

            growth_stages[ts] = current_stage
            transition_infos[ts] = transition_info
            previous_stage = current_stage

        print(f"Growth stages classified: {growth_stages}")

        # Stress detection for each timestamp
        for ts, ts_date in sorted_timestamps:
            if any(result.get("timestamp") == ts for result in multi_index_stress_results):
                continue

            # Extract the actual timestamp without the prefix for lookup in standardized_images
            # ts format: 'NDVI_2025-05-31_to_2025-06-06'
            # standardized_images key format: '2025-05-31_to_2025-06-06'
            base_timestamp = extract_timestamp_indices(ts)  # Extract just the date part

            print(f"Looking up timestamp: '{base_timestamp}' in standardized_images")

            # Retrieve full NDVI / index arrays from standardized_images for this timestamp
            index_data_current = {key: standardized_images[key].get(base_timestamp, (None, None))[0] for key in index_keys}
            data_status = [(k, 'None' if v is None else ('all NaN' if isinstance(v, np.ndarray) and np.isnan(v).all() else 'valid')) for k,v in index_data_current.items()]
            print(f"Index data for timestamp {ts}: {data_status}")

            ndvi_data = index_data_current["NDVI"]
            if ndvi_data is None or (isinstance(ndvi_data, np.ndarray) and np.isnan(ndvi_data).all()):
                print(f"WARNING: NDVI data is None or all NaN for timestamp: {ts}. Skipping stress detection.")
                continue
            else:
                print(f"NDVI data for timestamp {ts} is valid, shape: {ndvi_data.shape}")

            current_stage = growth_stages.get(ts, "Unknown")
            transition_info = transition_infos.get(ts)
            if transition_info is None:
                transition_info = {
                    "primary": current_stage,
                    "secondary": None,
                    "is_transition": False,
                    "confidence_scores": {current_stage: 1.0},
                    "raw_scores": {}
                }

            # Calculate stress using the correct transition info and crop type
            stress_percentage, stress_details = calculate_real_time_stress_percentage(
                index_data_current, transition_info, crop_type=crop_type
            )

            stress_type = classify_stress_type(
                index_data_current,
                transition_info,
                threshold=deviant_threshold,
                crop_type=crop_type
            )

            # FIXED: Use base_timestamp to construct the correct keys for mean values
            multi_index_stress_results.append({
                "timestamp": ts,
                "growth_stage": current_stage,
                "stress_percentage": stress_percentage,
                "stress_type": stress_type,
                "stress_details": stress_details,
                "mean_ndvi": index_data.get("NDVI", {}).get(f"NDVI_{base_timestamp}"),
                "mean_ndmi": index_data.get("NDMI", {}).get(f"NDMI_{base_timestamp}"),
                "mean_psri": index_data.get("PSRI", {}).get(f"PSRI_{base_timestamp}"),
                "mean_msavi": index_data.get("MSAVI", {}).get(f"MSAVI_{base_timestamp}"),
                "mean_gci": index_data.get("GCI", {}).get(f"GCI_{base_timestamp}"),
                "mean_ndre": index_data.get("NDRE", {}).get(f"NDRE_{base_timestamp}"),
                "mean_sipi": index_data.get("SIPI", {}).get(f"SIPI_{base_timestamp}")
            })

        multi_index_stress_results.sort(key=lambda x: x["timestamp"], reverse=True)

        # Return stress results, growth stages, and date cycle info
        return multi_index_stress_results, growth_stages, (sowing_date_ts, harvest_start, harvest_end, harvested_date)

    except Exception as e:
        print(f"Error in process_crop_monitoring_with_cycle_analysis: {str(e)}")
        # Return default values or re-raise the exception based on your needs
        return multi_index_stress_results or [], {}, (sowing_date, harvest_start, harvest_end, harvested_date)

############################################################## Yield_Calculation ############################################################

def transform_stress_data(stress_list, index_data, mean_ndvi_values=None):
    """
    Transform list of stress data into required dictionary format, adding mean_ndvi if missing

    Args:
        stress_list: List of dictionaries containing stress data
        index_data: Dictionary containing index data with 'NDVI' key
        mean_ndvi_values: Dictionary of mean NDVI values keyed by timestamp (fallback data source)

    Returns:
        Dictionary with timestamp keys and nested dictionaries containing mean_ndvi and stress percentage
        Returns None if stress_list is empty or None
    """
    # Check if stress data is empty or None
    if not stress_list:
        logging.warning("stress_list is empty or None, returning None.")
        return None

    # Determine which NDVI data to use
    if index_data and 'NDVI' in index_data and index_data['NDVI'] and len(index_data['NDVI']) > 0:
        selected_ndvi_values = index_data['NDVI']
    else:
        if mean_ndvi_values is None or len(mean_ndvi_values) == 0:
            logging.error("index_data['NDVI'] is empty and mean_ndvi_values is also empty or None.")
            # Use empty dict as fallback to avoid errors
            selected_ndvi_values = {}
        else:
            logging.info("index_data['NDVI'] is empty, using mean_ndvi_values as fallback.")
            selected_ndvi_values = mean_ndvi_values

    # First, let's add mean_ndvi to the stress_list if it's missing
    enriched_stress_list = []
    for item in stress_list:
        enriched_item = item.copy()  # Create a copy to avoid modifying original data
        if 'mean_ndvi' not in enriched_item:
            enriched_item['mean_ndvi'] = selected_ndvi_values.get(item['timestamp'], 0)
        enriched_stress_list.append(enriched_item)

    # Now transform the enriched data
    return {
        item['timestamp']: {
            'mean_ndvi': item['mean_ndvi'],
            'ndvi_stress_percentage': item['stress_percentage']
        }
        for item in enriched_stress_list
    }

class SimpleYieldEstimator:
    """
    Simplified yield estimation system focusing on core metrics
    and essential calculations.
    """

    def __init__(self):
        # Initialize crop coefficients organized by country
        self.crop_coefficients = {
            'India': {
                'Rice': {'base_yield': 2.8, 'max_yield': 4.5, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.4, 'seasonal_factors': {'Kharif': 1.0, 'Rabi': 0.95, 'summer': 0.8},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 60, 'weight': 0.3}, 'reproductive': {'start': 61, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 0.8},
                'Cotton': {'base_yield': 2, 'max_yield': 3.5, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.6, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.8, 'summer': 0.9},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 60, 'weight': 0.3}, 'flowering': {'start': 61, 'end': 110, 'weight': 0.4}, 'boll_development': {'start': 111, 'end': 160, 'weight': 0.3}},
                        'minimum_yield_factor': 0.8},
                'Sanghar_White_Onion': {'base_yield': 35, 'max_yield': 60, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.45, 'seasonal_factors': {'kharif': 0.85, 'rabi': 1.0, 'summer': 0.8},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'bulb_formation': {'start': 46, 'end': 95, 'weight': 0.4}, 'maturity': {'start': 96, 'end': 115, 'weight': 0.3}},
                        'minimum_yield_factor': 0.8},
                'White_Onion': {'base_yield': 25, 'max_yield': 45, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.45, 'seasonal_factors': {'kharif': 0.85, 'rabi': 1.0, 'summer': 0.8},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'bulb_formation': {'start': 46, 'end': 95, 'weight': 0.4}, 'maturity': {'start': 96, 'end': 115, 'weight': 0.3}},
                        'minimum_yield_factor': 0.8},
                'Wheat': {'base_yield': 4, 'max_yield': 5.5, 'ndvi_threshold': 0.7, 'stress_tolerance': 0.5, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.85, 'summer': 0.75},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'reproductive': {'start': 46, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 0.8},
                'Gram': {'base_yield': 2.5, 'max_yield': 3.5, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.6, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.8, 'summer': 0.9},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'reproductive': {'start': 46, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 0.8},
                'Potato': {'base_yield': 20, 'max_yield': 35, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.45, 'seasonal_factors': {'kharif': 0.85, 'rabi': 1.0, 'summer': 0.8},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'reproductive': {'start': 46, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 0.8},
                'Red_Onion': {'base_yield': 30, 'max_yield': 40, 'ndvi_threshold': 0.7, 'stress_tolerance': 0.5, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.85, 'summer': 0.75},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 35, 'weight': 0.3}, 'bulb_formation': {'start': 36, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 130, 'weight': 0.3}},
                        'minimum_yield_factor': 0.8},
                'Maize': {'base_yield': 4.5, 'max_yield': 8.5, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.6, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.8, 'summer': 0.9},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'reproductive': {'start': 46, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 0.8},
                'Soyabean': {'base_yield': 1.5, 'max_yield': 3.5, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.45, 'seasonal_factors': {'kharif': 0.85, 'rabi': 1.0, 'summer': 0.8},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'reproductive': {'start': 46, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 0.9},
                'Mustard': {'base_yield': 1.5, 'max_yield': 4.0, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.6, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.8, 'summer': 0.9},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'reproductive': {'start': 46, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 0.9}
            },

            'Slovakia': {
                'Rice': {'base_yield': 3.5, 'max_yield': 5.5, 'ndvi_threshold': 0.7, 'stress_tolerance': 0.5, 'seasonal_factors': {'Kharif': 1.0, 'rabi': 0.95, 'summer': 0.8},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 60, 'weight': 0.3}, 'reproductive': {'start': 61, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0},
                'Wheat': {'base_yield': 6.0, 'max_yield': 9.2, 'ndvi_threshold': 0.7, 'stress_tolerance': 0.5, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.85, 'summer': 0.75},
                        'growth_stages': {"vegetative": {"start": 0, "end": 78, "weight": 0.3}, "reproductive": {"start": 79, "end": 182, "weight": 0.4}, "maturity": {"start": 183, "end": 260, "weight": 0.3}},
                        'minimum_yield_factor': 1.0},
                'Maize': {'base_yield': 6.5, 'max_yield': 10.5, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.6, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.8, 'summer': 0.9},
                        'growth_stages': {"vegetative": {"start": 0, "end": 65, "weight": 0.3}, "reproductive": {"start": 66, "end": 151, "weight": 0.4}, "maturity": {"start": 152, "end": 215, "weight": 0.3}},
                        'minimum_yield_factor': 1.0},
                'Soyabean': {'base_yield': 1.8, 'max_yield': 2.4, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.45, 'seasonal_factors': {'kharif': 0.85, 'rabi': 1.0, 'summer': 0.8},
                        'growth_stages': {"vegetative": {"start": 0, "end": 47, "weight": 0.3}, "reproductive": {"start": 48, "end": 108, "weight": 0.4}, "maturity": {"start": 109, "end": 155, "weight": 0.3}},
                        'minimum_yield_factor': 1.0},
                'Barley': {'base_yield': 4.0, 'max_yield': 6.8, 'ndvi_threshold': 0.7, 'stress_tolerance': 0.5, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.9, 'summer': 0.8},
                        'growth_stages': {"vegetative": {"start": 0, "end": 45, "weight": 0.3}, "reproductive": {"start": 46, "end": 105, "weight": 0.4}, "maturity": {"start": 106, "end": 150, "weight": 0.3}},
                        'minimum_yield_factor': 1.0},
                'Potato': {'base_yield': 30, 'max_yield': 50, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.45, 'seasonal_factors': {'kharif': 0.85, 'rabi': 1.0, 'summer': 0.8},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'tuber_formation': {'start': 46, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0},
                'Sunflower': {'base_yield': 1.5, 'max_yield': 2.5, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.55, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.9, 'summer': 0.85},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 40, 'weight': 0.3}, 'flowering': {'start': 41, 'end': 80, 'weight': 0.4}, 'maturity': {'start': 81, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0},
                'Rapeseed': {'base_yield': 2.0, 'max_yield': 3.5, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.5, 'seasonal_factors': {'kharif': 1.0, 'rabi': 0.9, 'summer': 0.85},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 50, 'weight': 0.3}, 'reproductive': {'start': 51, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0}
            },

            'Philippines': {
                'Rice': {'base_yield': 3.2, 'max_yield': 4.7, 'ndvi_threshold': 0.7, 'stress_tolerance': 0.5, 'seasonal_factors': {'rabi': 1.0, 'summer': 0.85},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 50, 'weight': 0.3}, 'reproductive': {'start': 51, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0},
                'Maize': {'base_yield': 5.5, 'max_yield': 8.5, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.6, 'seasonal_factors': {'rabi': 1.0, 'summer': 0.8},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 40, 'weight': 0.3}, 'reproductive': {'start': 41, 'end': 80, 'weight': 0.4}, 'maturity': {'start': 81, 'end': 110, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0}
            },

            'Netherlands': {
                'White_Onion': {'base_yield': 25, 'max_yield': 45, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.45, 'seasonal_factors': {'kharif': 0.85, 'rabi': 1.0, 'summer': 0.8},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'bulb_formation': {'start': 46, 'end': 95, 'weight': 0.4}, 'maturity': {'start': 96, 'end': 115, 'weight': 0.3}},
                        'minimum_yield_factor': 0.8},
            },

            'Egypt': {
                'Potato': {'base_yield': 15, 'max_yield': 20, 'ndvi_threshold': 0.75, 'stress_tolerance': 0.45, 'seasonal_factors': {'rabi': 1.0, 'summer': 0.85},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 40, 'weight': 0.3}, 'tuber_formation': {'start': 41, 'end': 80, 'weight': 0.4}, 'maturity': {'start': 81, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0},
                'Rice': {'base_yield': 4.5, 'max_yield': 6.0, 'ndvi_threshold': 0.7, 'stress_tolerance': 0.5, 'seasonal_factors': {'rabi': 1.0, 'summer': 0.9},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 50, 'weight': 0.3}, 'reproductive': {'start': 51, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0},
                'Wheat': {'base_yield': 3.5, 'max_yield': 4.5, 'ndvi_threshold': 0.7, 'stress_tolerance': 0.5, 'seasonal_factors': {'rabi': 1.0, 'summer': 0.85},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'reproductive': {'start': 46, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0},
                'Cotton': {'base_yield': 1.0, 'max_yield': 2.0, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.55, 'seasonal_factors': {'rabi': 1.0, 'summer': 0.9},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 50, 'weight': 0.3}, 'flowering': {'start': 51, 'end': 80, 'weight': 0.4}, 'boll_development': {'start': 81, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0},
                'Maize': {'base_yield': 3.0, 'max_yield': 5.0, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.6, 'seasonal_factors': {'rabi': 1.0, 'summer': 0.85},
                        'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'reproductive': {'start': 46, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
                        'minimum_yield_factor': 1.0}
            }
        }

        # Default coefficients for unknown crops
        self.default_coefficients = {
            'base_yield': 30, 'max_yield': 50, 'ndvi_threshold': 0.65, 'stress_tolerance': 0.5,
            'seasonal_factors': {'kharif': 1.0, 'rabi': 1.0, 'summer': 0.9},
            'growth_stages': {'vegetative': {'start': 0, 'end': 45, 'weight': 0.3}, 'reproductive': {'start': 46, 'end': 90, 'weight': 0.4}, 'maturity': {'start': 91, 'end': 120, 'weight': 0.3}},
            'minimum_yield_factor': 0.8
        }

    from datetime import datetime
    from typing import Union


    def _convert_sowing_date(self, sowing_date: Union[str, datetime]) -> datetime:
        """
        Convert sowing date to datetime object if it's a string.

        Parameters:
            sowing_date: Date in string format or datetime object

        Returns:
            datetime object
        """
        if isinstance(sowing_date, str):
            try:
                # Try common date formats
                date_formats = [
                    "%Y-%m-%d %H:%M:%S",
                    "%Y-%m-%d",
                    "%d-%m-%Y",
                    "%Y/%m/%d",
                    "%d/%m/%Y"
                ]

                for date_format in date_formats:
                    try:
                        return datetime.strptime(sowing_date, date_format)
                    except ValueError:
                        continue

                raise ValueError(f"Unable to parse date string: {sowing_date}")

            except Exception as e:
                raise ValueError(f"Error converting sowing date: {str(e)}")

        elif isinstance(sowing_date, datetime):
            return sowing_date
        else:
            raise ValueError("Sowing date must be either a string or datetime object")

    def _process_data(self, stress_data: Dict, sowing_date) -> Dict:
        """Process and validate input data."""
        try:
            ndvi_values = []
            stress_values = []
            dates = []

            for date_key, data in stress_data.items():
                if not isinstance(data, dict) or 'mean_ndvi' not in data or 'ndvi_stress_percentage' not in data:
                    raise ValueError(f"Invalid data format for date {date_key}")

                # Handle date keys with different formats
                if isinstance(date_key, str) and "_to_" in date_key:
                    date_str = date_key.split('_to_')[1]
                    date = datetime.strptime(date_str, '%Y-%m-%d')
                else:
                    # Try to parse the date directly
                    try:
                        date = datetime.strptime(date_key, '%Y-%m-%d')
                    except ValueError:
                        # If it fails, try to use the date key as is if it's a datetime object
                        date = date_key if isinstance(date_key, datetime) else None
                        if date is None:
                            raise ValueError(f"Unable to parse date from key: {date_key}")

                ndvi_values.append(float(data['mean_ndvi']))
                stress_values.append(float(data['ndvi_stress_percentage']))
                dates.append(date)

            processed_data = {
                'ndvi_values': np.array(ndvi_values),
                'stress_values': np.array(stress_values),
                'reading_count': len(ndvi_values)
            }

            if sowing_date is not None and dates:
                # Convert sowing_date to datetime if it's a Timestamp
                if hasattr(sowing_date, 'to_pydatetime'):
                    sowing_date = sowing_date.to_pydatetime()

                # Sort dates for correct calculation
                dates.sort()

                # Use the last date that's within the valid range
                last_valid_date = dates[-1]
                days_after_sowing = (last_valid_date - sowing_date).days
                processed_data['days_after_sowing'] = max(0, days_after_sowing)

            return processed_data

        except Exception as e:
            raise ValueError(f"Error processing data: {str(e)}")

    def _calculate_ndvi_impact(self, ndvi_values: np.ndarray, threshold: float) -> float:
        """
        Calculate NDVI impact with dynamic scaling based on growth stage patterns.
        """
        if len(ndvi_values) == 0:
            return 1.0

        # Calculate NDVI progression
        weights = np.linspace(0.5, 1.0, len(ndvi_values))
        weighted_ndvi = np.average(ndvi_values, weights=weights)

        # Check for positive trend in recent values
        recent_trend = np.polyfit(range(min(5, len(ndvi_values))),
                                ndvi_values[-min(5, len(ndvi_values)):], 1)[0]

        # Adjust threshold based on growth stage
        adjusted_threshold = threshold * 0.7  # More realistic threshold
        ndvi_ratio = weighted_ndvi / adjusted_threshold

        # Calculate base impact with more generous scaling
        base_impact = 0.8 + (ndvi_ratio - 0.6) * 0.8

        # Add bonus for positive trend
        trend_bonus = 0.1 if recent_trend > 0 else 0

        final_impact = base_impact + trend_bonus
        return max(0.8, min(1.5, final_impact))  # Allow higher maximum impact


    def _calculate_stress_impact(self, stress_values: np.ndarray, tolerance: float) -> float:
        """
        Calculate stress impact with consideration for healthy stress levels.
        """
        if len(stress_values) == 0:
            return 1.0

        # Consider recent stress levels
        recent_stress = np.mean(stress_values[-3:])

        # Define healthy stress range
        optimal_stress_range = (15, 25)  # Define what's considered healthy

        if optimal_stress_range[0] <= recent_stress <= optimal_stress_range[1]:
            # Bonus for being in healthy range
            base_impact = 1.2
        else:
            # Gradually reduce impact outside optimal range
            distance_from_optimal = min(
                abs(recent_stress - optimal_stress_range[0]),
                abs(recent_stress - optimal_stress_range[1])
            )
            base_impact = 1.2 - (distance_from_optimal / 100)

        # Consider stress consistency
        stress_std = np.std(stress_values)
        consistency_bonus = 0.1 if stress_std < 5 else 0  # Bonus for consistent stress

        final_impact = base_impact + consistency_bonus
        return max(0.8, min(1.4, final_impact))  # Allow higher maximum impact


    def _calculate_growth_stage_factor(self, days_after_sowing: int, growth_stages: Dict) -> float:
        """
        Calculate growth stage impact with more dynamic scaling.
        """
        if days_after_sowing is None:
            return 1.0

        stage_factor = 1.0
        for stage, info in growth_stages.items():
            if info['start'] <= days_after_sowing <= info['end']:
                # Calculate progress through current stage
                stage_progress = (days_after_sowing - info['start']) / (info['end'] - info['start'])

                # Add dynamic scaling based on stage
                if stage == 'vegetative':
                    # Reward good early growth
                    stage_factor = 0.9 + (stage_progress * info['weight'] * 1.5)
                elif stage == 'bulb_formation':
                    # Critical stage with highest potential impact
                    stage_factor = 1.0 + (stage_progress * info['weight'] * 1.8)
                else:  # maturity
                    # Maintain good results through maturity
                    stage_factor = 1.1 + (stage_progress * info['weight'])
                break

        return min(1.6, max(0.9, stage_factor))

    def _convert_date(self, date_value: Union[str, datetime]) -> datetime:
        print(f"Original date_value: {date_value!r}")
        if isinstance(date_value, str):
            # Extract date if extra text is present
            if "_to_" in date_value:
                date_value = date_value.split("_to_")[1]
                print(f"After splitting: {date_value!r}")

            # Now parse the date
            result = datetime.strptime(date_value, "%Y-%m-%d").date()
            print(f"Parsed result: {result}")
            return result
        return date_value

    def estimate_yield(
            self,
            crop_type: str,
            stress_data: Dict,
            sowing_date: Union[str, datetime],
            season: str,
            country: str,
            harvest_date: Union[str, datetime, None] = None
        ) -> Dict:
            """
            Estimate yield based on NDVI and stress factors, considering harvest timing.

            Parameters:
                crop_type: Type of crop
                stress_data: Dictionary containing NDVI and stress measurements
                sowing_date: Sowing date as string or datetime object
                season: Growing season ('kharif', 'rabi', 'summer')
                harvest_date: Optional harvest date as string or datetime object

            Returns:
                Dictionary containing yield estimate and confidence metrics
                Returns yield of 0 if stress_data is empty
            """
            if not isinstance(stress_data, dict) or not stress_data:
                print("Warning: stress_data is empty or invalid, returning yield of 0")
                return {
                    'estimated_yield': 0.0,
                    'confidence_score': 0.0,
                    'metrics': {
                        'ndvi_factor': 0.0,
                        'stress_impact': 0.0,
                        'seasonal_factor': 0.0,
                        'growth_stage_factor': 0.0,
                        'days_after_sowing': None,
                        'harvest_date': None
                    }
                }

            season = season.lower()
            if season not in ['kharif', 'rabi', 'summer']:
                raise ValueError("Invalid season. Must be 'kharif', 'rabi', or 'summer'")

            try:
                # Convert sowing and harvest dates to datetime
                sowing_date = self._convert_sowing_date(sowing_date)
                today = datetime.today()

                if harvest_date:
                    harvest_date = self._convert_date(harvest_date)
                else:
                    harvest_date = today  # Default to today if harvest date is not provided

                # Filter stress data to only include dates between sowing and harvest
                filtered_stress_data = {}

                for date_key, values in stress_data.items():
                    try:
                        # Extract the relevant date from keys formatted as "YYYY-MM-DD_to_YYYY-MM-DD"
                        if isinstance(date_key, str) and "_to_" in date_key:
                            parsed_date_key = date_key.split("_to_")[1]
                        else:
                            parsed_date_key = date_key

                        date_obj = datetime.strptime(parsed_date_key, "%Y-%m-%d")

                        # Ensure we only consider data between sowing and the correct harvest limit
                        if sowing_date <= date_obj <= (harvest_date if harvest_date < today else today):
                            filtered_stress_data[date_key] = values

                    except ValueError as e:
                        print(f"Warning: Skipping invalid date key '{date_key}', Error: {e}")

                if not filtered_stress_data:
                    print("Warning: No valid NDVI data found within the allowed date range, returning yield of 0")
                    return {
                        'estimated_yield': 0.0,
                        'confidence_score': 0.0,
                        'metrics': {
                            'ndvi_factor': 0.0,
                            'stress_impact': 0.0,
                            'seasonal_factor': 0.0,
                            'growth_stage_factor': 0.0,
                            'days_after_sowing': None,
                            'harvest_date': harvest_date.strftime("%Y-%m-%d")
                        }
                    }

                if country not in self.crop_coefficients:
                    raise ValueError(f"Country '{country}' not found in crop coefficients.")

                coeff = self.crop_coefficients[country].get(crop_type, self.default_coefficients)

                # Process input data
                processed_data = self._process_data(filtered_stress_data, sowing_date)

                # Compute yield factors
                growth_stage_factor = self._calculate_growth_stage_factor(
                    processed_data.get('days_after_sowing'),
                    coeff['growth_stages']
                )
                ndvi_factor = self._calculate_ndvi_impact(processed_data['ndvi_values'], coeff['ndvi_threshold'])
                stress_impact = self._calculate_stress_impact(processed_data['stress_values'], coeff['stress_tolerance'])
                seasonal_factor = coeff['seasonal_factors'].get(season, 1.0)

                # Calculate estimated yield
                base_yield = coeff['base_yield']
                estimated_yield = base_yield * ndvi_factor * stress_impact * seasonal_factor * growth_stage_factor

                # Apply min-max yield limits
                final_yield = np.clip(estimated_yield, base_yield * coeff['minimum_yield_factor'], coeff['max_yield'])

                # Compute confidence score
                confidence = self._calculate_confidence(processed_data, True)

                return {
                    'estimated_yield': round(final_yield, 2),
                    'confidence_score': confidence,
                    'metrics': {
                        'ndvi_factor': round(ndvi_factor, 3),
                        'stress_impact': round(stress_impact, 3),
                        'seasonal_factor': round(seasonal_factor, 3),
                        'growth_stage_factor': round(growth_stage_factor, 3),
                        'days_after_sowing': processed_data.get('days_after_sowing', None),
                        'harvest_date': harvest_date.strftime("%Y-%m-%d")
                    }
                }

            except Exception as e:
                raise ValueError(f"Error in yield estimation: {str(e)}")

    def _calculate_confidence(self, data: Dict, has_sowing_date: bool) -> float:
        """
        Calculate confidence score with more balanced criteria.
        """
        # Base confidence from number of readings
        base_confidence = min(data['reading_count'] / 3, 1.0)  # Reduced required readings

        # Adjust for data consistency
        consistency = 1.0
        if data['reading_count'] > 1:
            consistency = 1 - min(np.std(data['ndvi_values']) / np.mean(data['ndvi_values']), 0.5)

        # Adjust for sowing date availability
        sowing_date_factor = 1.2 if has_sowing_date else 1.0

        confidence = (base_confidence * 0.6 + consistency * 0.4) * sowing_date_factor
        return round(min(confidence * 100, 100), 1)

################################################# Upload To MongoDB ######################################################
# Utility Function to Extract Timestamp from File Path
def extract_timestamp(file_path):
    """Extracts the date range from the file path."""
    match = re.search(r'(\d{4}-\d{2}-\d{2}_to_\d{4}-\d{2}-\d{2})', file_path)
    return match.group(1) if match else file_path  # Return file path if no date found

def process_tiff_structure(file_data):
    """
    Process TIFF files to extract data, CRS, transform, height, and width.
    Convert data to Base64 format for storage.

    Args:
        file_data (dict): A dictionary with file paths as keys and their metadata as values.

    Returns:
        list: A list of dictionaries with processed TIFF data and metadata.
    """
    result = []

    # Process each file in the dictionary
    for file_path, content in file_data.items():
        # Validate the content
        if not isinstance(content, dict):
            logging.error(f"Content is not a dictionary for file: {file_path}")
            continue

        # Extract 'data' key
        data = content.get('data')
        if data is None:
            logging.error(f"'data' key missing in file: {file_path}")
            continue

        try:
            # Convert data to Base64
            data_base64 = base64.b64encode(data.tobytes()).decode('utf-8')

            # Extract CRS, transform, height, and width
            crs = content.get('crs')
            transform = content.get('transform')
            height = content.get('height')
            width = content.get('width')

            # Log a warning if CRS, transform, height, or width is missing
            if crs is None or transform is None:
                logging.warning(f"'crs' or 'transform' missing for file: {file_path}")

            if height is None or width is None:
                logging.warning(f"'height' or 'width' missing for file: {file_path}")

            # Append processed file information
            result.append({
                'filename': os.path.basename(file_path),
                'data_base64': data_base64,
                'crs': crs,
                'transform': transform,
                'height': height,
                'width': width
            })

        except Exception as e:
            # Log any errors during processing
            logging.error(f"Error processing file {file_path}: {e}")

    return result

# Function to Process All Indices and Plant Height
def process_all_indices_and_plant_height(organized_images):
    """
    Process all indices and plant height images from the organized TIFF files.

    Args:
        organized_images (dict): A dictionary grouped by indices or 'plant_height'.

    Returns:
        dict: A dictionary with processed results for each category.
    """
    processed_results = {}

    # Iterate through each category (e.g., NDVI, NDMI, plant_height)
    for category, files in organized_images.items():
        print(f"Processing category: {category}")

        # Process all TIFF files in the category
        processed_files = process_tiff_structure(files)

        # Add the category to each file's metadata
        for file in processed_files:
            file['category'] = category

        # Store the processed files in the results dictionary
        processed_results[category] = processed_files

    return processed_results

def upload_to_mongo(url, mobile, farm_name, start_date, end_date, crop_season, area,
                    latest_timestamp, index_data,
                    organized_images, multi_index_stress_results, sowing_date, harvest_window_start, harvest_window_end, harvested_date, yield_data, growth_stages):
    """
    Upload the predicted data to MongoDB using the provided API.

    Args:
        url (str): API endpoint to upload data.
        mobile (int): Mobile number.
        farm_name (str): Farm name.
        start_date (datetime or str): Start date.
        end_date (datetime or str): End date.
        crop_season (str): Crop season.
        area (float): Area of the farm.
        organized_images (dict): Organized TIFF images grouped by index or plant height.
        overall_health (float): Health score of the farm.
        performance_category (str): Health category of the farm.
        overall_stress (float): Overall stress percentage.
        multi_index_stress_results (dict): Stress results for each image with percentages and types.
        yield_data (dict): Yield-related information.

    Returns:
        response: API response.
    """

    # Convert dates and overall_stress to strings if needed
    start_date = str(start_date)
    print(f"start_date: {start_date}")
    # if isinstance(end_date, datetime):
    #     end_date = end_date.strftime('%Y-%m-%d'
    # end_date = end_date.date()
    end_date = str(end_date)
    latest_timestamp = str(latest_timestamp)

    # Simplify multi_index_stress_results by extracting timestamp, stress percentage, and stress type
    simplified_stress_results = []
    for entry in multi_index_stress_results:
        timestamp = entry.get('timestamp')
        mean_ndvi = entry.get('mean_ndvi')
        stress_percentage = entry.get('stress_percentage')
        stress_type = entry.get('stress_type')

        # Extract only the date range from the timestamp
        clean_timestamp = timestamp.split('_', 1)[1] if timestamp and '_' in timestamp else timestamp

        simplified_stress_results.append({
            "timestamp": clean_timestamp,
            "mean_ndvi": mean_ndvi,
            "stress_percentage": stress_percentage,
            "stress_type": stress_type
        })

    # Process TIFF images for all indices and plant height
    processed_tiff_data = process_all_indices_and_plant_height(organized_images)

    # Flatten processed_tiff_data into a list
    flattened_tiff_data = []
    for category, files in processed_tiff_data.items():
        flattened_tiff_data.extend(files)

    # Extract yield-related data
    sowing_date = str(sowing_date)
    harvest_window_start = str(harvest_window_start)
    harvest_window_end = str(harvest_window_end)
    harvested_date = str(harvested_date)
    crop_yield = str(yield_data.get('estimated_yield', 'N/A'))

    # Create harvest window string
    harvest_window = f"{harvest_window_start}_to_{harvest_window_end}"

    # Prepare data payload with simplified multi_index_stress_results
    data = {
        "mobile": mobile,
        "farm_name": farm_name,
        "start_date": start_date,
        "end_date": end_date,
        "crop_season": crop_season,
        "area": area,
        "latest_timestamp": latest_timestamp,
        "mean_ndvi_values": index_data['NDVI'],  # Ensure correct field name
        "mean_ndmi_values": index_data['NDMI'],
        "mean_psri_values": index_data['PSRI'],
        "mean_gci_values": index_data['GCI'],
        "mean_msavi_values": index_data['MSAVI'],
        "mean_ndwi_values": index_data['NDWI'],
        "mean_sipi_values": index_data['SIPI'],
        "mean_ndre_values": index_data['NDRE'],
        "tiff_data": flattened_tiff_data,
        "stress_results_data": simplified_stress_results,
        "harvest_window": harvest_window,
        "crop_yield": crop_yield,
        "sowing_date": sowing_date,
        "harvest_date": harvested_date,
        "growth_stages": growth_stages
    }

    # Log the simplified data payload
    logging.info("Simplified Data payload for MongoDB upload:")
    logging.info(json.dumps(data, indent=4))

    headers = {'Content-Type': 'application/json'}

    try:
        response = requests.post(url, json=data, headers=headers)

        # Print response details
        print("\nMongoDB API Response for simplified payload:")
        print(f"Status Code: {response.status_code}")
        print(f"Response Text: {response.text}")

        if response.status_code == 200:
            print("Simplified data successfully uploaded to MongoDB.")
        else:
            print("Failed to upload simplified data to MongoDB.")

        return response
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during MongoDB upload: {e}")
        return None

############################################## Disease and Pest infestation probability analysis ###########################################

class PestDiseasePredictor:
    def __init__(self, crop_pest_disease_library):
        from datetime import datetime
        from typing import Any, Dict, Optional
        self.library = crop_pest_disease_library
        self.crop_growth_stages = {
            'Bajra': {'Sowing': 0,'Germination': 10,'Tillering': 25,'Booting': 50,'Flowering': 60,'Grain Filling': 70,'Maturity': 85,'Harvest': 110},
            'Cotton': {'Sowing': 0,'Germination': 10,'Vegetative': 25,'Square': 35,'Flowering': 45,'Boll Formation': 75,'Maturity': 105,'Harvest': 150},
            'Gram': {'Sowing': 0,'Germination': 10,'Branching': 30,'Vegetative': 25,'Flowering': 40,'Pod Filling': 50,'Maturity': 70,'Harvest': 90},
            'Groundnut': {'Sowing': 0,'Germination': 10,'Vegetative': 25,'Pegging': 35,'Flowering': 45,'Pod Filling': 70,'Maturity': 85,'Harvest': 105},
            'Maize': { 'Sowing': 0,'Germination': 10,'Vegetative': 25,'Tasseling': 45,'Silking': 55,'Grain Filling': 70,'Maturity': 85,'Harvest': 100},
            'Mustard': { 'Sowing': 0,'Germination': 10,'Rosette': 25,'Elongation': 40,'Flowering': 60,'Pod Formation': 75,'Maturity': 85,'Harvest': 100},
            'White_Onion': {'Sowing': 0,'Germination': 15,'Transplanting': 35,'Vegetative': 75,'Bulb Initiation': 90,'Bulb Enlargement': 105,'Maturity': 110,'Harvest': 140},
            'Red_Onion': {'Sowing': 0,'Germination': 15,'Transplanting': 35,'Vegetative': 75,'Bulb Initiation': 90,'Bulb Enlargement': 105,'Maturity': 110,'Harvest': 140},
            'Potato': {'Sowing': 0,'Germination': 15,'Vegetative': 30,'Tuber Initiation': 40,'Tuber Bulking': 70,'Maturity': 85,'Harvest': 105},
            'Rice': {'Sowing': 0,'Germination': 10,'Tillering': 30,'Booting': 75,'Panicle Initiation': 60,'Flowering': 90,'Grain Filling': 100, 'Maturity': 110,'Harvest': 125},
            'Soyabean': {'Sowing': 0,'Germination': 10,'Vegetative': 35,'Flowering': 50,'Pod Development': 60,'Seed Filling': 65,'Maturity': 70,'Harvest': 85},
            'Wheat': {'Sowing': 0,'Germination': 15,'Tillering': 45,'Elongation': 60,'Heading': 75,'Grain Filling': 95,'Maturity': 105,'Harvest': 120},
            'Chilli': {'Sowing': 0,'Germination': 10,'Transplanting': 30,'Vegetative': 50,'Flowering': 65,'Fruiting': 80,'Maturity': 100,'First Picking': 105,'Second Maturity': 150,'Second Picking': 165,'Harvest': 180},
            'Tur': {'Sowing': 0,'Germination': 10,'Vegetative': 30,'Flowering': 45,'Pod Development': 55,'Pod Filling': 65,'Maturity': 85,'Harvest': 115},
            'Banana': {'Planting': 0, 'Establishment': 30,'Vegetative_Growth': 90,'Pseudostem_Development': 150,'Flower_Initiation': 210,'Bunch_Emergence': 240,'Fruit_Development': 270,'Fruit_Filling': 300,'Maturity': 330,'Harvest': 365}
        }

    def _parse_date(self, date_str):
        """Parse date string to datetime object using multiple formats."""
        from datetime import datetime

        if isinstance(date_str, datetime):
            return date_str

        date_formats = ["%Y-%m-%d", "%d-%m-%Y", "%m/%d/%Y", "%d/%m/%Y", "%Y/%m/%d", "%Y-%m-%d %H:%M:%S"]

        for date_format in date_formats:
            try:
                return datetime.strptime(date_str, date_format)
            except (ValueError, TypeError):
                continue

        return None

    def _calculate_crop_age(self, sowing_date: str, observation_date=None) -> int:
        """
        Calculate the age of the crop in days from sowing date to observation date.

        Args:
            sowing_date: Date when the crop was sown
            observation_date: Date for which to calculate the crop age (defaults to today)

        Returns:
            Age of the crop in days (integer)
        """
        from datetime import datetime

        if not sowing_date:
            return 0

        try:
            # Parse sowing date
            parsed_sowing_date = self._parse_date(sowing_date)
            if not parsed_sowing_date:
                return 0

            # Use provided observation date or default to today
            if observation_date:
                parsed_observation_date = self._parse_date(observation_date)
                if not parsed_observation_date:
                    parsed_observation_date = datetime.today()
            else:
                parsed_observation_date = datetime.today()

            # Calculate days since sowing (crop age)
            days_since_sowing = (parsed_observation_date - parsed_sowing_date).days

            return max(0, days_since_sowing)  # Ensure age is not negative

        except Exception as e:
            print(f"Error calculating crop age: {e}")
            return 0

    def _get_growth_stage(self, sowing_date: str, crop: str, observation_date=None) -> str:
        """
        Determine the growth stage of the crop based on sowing date and observation date.

        Args:
            sowing_date: Date when the crop was sown
            crop: Type of crop
            observation_date: Date for which to determine the growth stage (defaults to today)

        Returns:
            Growth stage of the crop
        """
        from datetime import datetime

        if not sowing_date or crop not in self.crop_growth_stages:
            return "Unknown"

        try:
            # Parse sowing date
            parsed_sowing_date = self._parse_date(sowing_date)
            if not parsed_sowing_date:
                return "Unknown"

            # Use provided observation date or default to today
            if observation_date:
                parsed_observation_date = self._parse_date(observation_date)
                if not parsed_observation_date:
                    parsed_observation_date = datetime.today()
            else:
                parsed_observation_date = datetime.today()

            # Calculate days since sowing
            days_since_sowing = (parsed_observation_date - parsed_sowing_date).days

            if days_since_sowing < 0:
                return "Pre-Sowing"  # Future sowing date relative to observation

            growth_stages = self.crop_growth_stages[crop]
            current_stage = "Sowing"  # Default to sowing

            # Find the right stage based on days since sowing
            for stage, days in sorted(growth_stages.items(), key=lambda x: x[1]):
                if days_since_sowing >= days:
                    current_stage = stage

            return current_stage

        except Exception as e:
            print(f"Error determining growth stage: {e}")
            return "Unknown"

    def fetch_weather_data(self, url: str, payload: dict) -> pd.DataFrame:
        import requests
        import pandas as pd

        try:
            response = requests.post(url, json=payload)
            response.raise_for_status()
            data = response.json()
            if "weather_data" in data and isinstance(data["weather_data"], list):
                return pd.DataFrame(data["weather_data"])
            else:
                return pd.DataFrame([data])
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return pd.DataFrame()

    def process_data(self, df: pd.DataFrame, sowing_date=None):
        import pandas as pd

        if df.empty:
            return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

        row = df.iloc[0]
        farm_data = row.get('result', {})
        weather_data = row.get('weather', {})

        # Use the provided sowing_date parameter if available
        actual_sowing_date = sowing_date if sowing_date else farm_data.get('sowing_date', '')

        # Create farm dataframe with basic info
        farm_df = pd.DataFrame([{
            'farm_id': farm_data.get('_id', ''),
            'mobile': farm_data.get('mobile', ''),
            'farm_name': farm_data.get('farm_name', ''),
            'sowing_date': actual_sowing_date,
            'crop': farm_data.get('crop', ''),
            'harvest_date': farm_data.get('harvest_date', '')
        }])

        # Extract weather data
        weather_df = pd.DataFrame([{
            'date': day.get('date', ''),
            'maxtemp_c': day.get('day', {}).get('maxtemp_c', ''),
            'mintemp_c': day.get('day', {}).get('mintemp_c', ''),
            'avgtemp_c': day.get('day', {}).get('avgtemp_c', ''),
            'maxwind_kph': day.get('day', {}).get('maxwind_kph', ''),
            'totalprecip_mm': day.get('day', {}).get('totalprecip_mm', ''),
            'avghumidity': day.get('day', {}).get('avghumidity', ''),
        } for day in weather_data.get('allData', [])])

        # Create merged dataframe
        if not weather_df.empty and not farm_df.empty:
            # Create merged dataframe with all weather and farm data
            merged_df = weather_df.copy()
            farm_row_dict = farm_df.iloc[0].to_dict()

            # Apply farm data to each row in weather_df
            for key, value in farm_row_dict.items():
                merged_df[key] = value

            # Add growth stage and crop age for each date in weather data
            if 'crop' in merged_df.columns and 'sowing_date' in merged_df.columns:
                merged_df['growth_stage'] = merged_df.apply(
                    lambda row: self._get_growth_stage(
                        row['sowing_date'],
                        row['crop'],
                        row['date']
                    ),
                    axis=1
                )

                # Add crop age for each date
                merged_df['crop_age_days'] = merged_df.apply(
                    lambda row: self._calculate_crop_age(
                        row['sowing_date'],
                        row['date']
                    ),
                    axis=1
                )
        else:
            merged_df = farm_df
            if not merged_df.empty and 'crop' in merged_df.columns and 'sowing_date' in merged_df.columns:
                merged_df['growth_stage'] = self._get_growth_stage(
                    merged_df.iloc[0]['sowing_date'],
                    merged_df.iloc[0]['crop']
                )

                # Add crop age for current date
                merged_df['crop_age_days'] = self._calculate_crop_age(
                    merged_df.iloc[0]['sowing_date']
                )

        return farm_df, weather_df, merged_df

    def calculate_score(self, farm_value, min_value, max_value):
        """Calculate closeness score from pest optimal range (1 = center, 0 = outside)"""
        if min_value is None or max_value is None or farm_value is None:
            return 0

        center = (min_value + max_value) / 2
        half_range = (max_value - min_value) / 2

        if half_range == 0:
            return 1 if farm_value == center else 0

        distance = abs(farm_value - center)
        normalized_distance = distance / half_range

        # Ensure score doesn't exceed 1.0
        return max(0, min(1, 1 - normalized_distance))  # score between 0 and 1

    def growth_stage_match_score(self, current_stage, pest_preferred_stages):
        """
        Calculate how well the current growth stage matches the pest's preferred stages.
        Returns a score between 0-1 and the name of the matched stage.
        """
        if not current_stage or not pest_preferred_stages:
            return 0, "No match"

        # If pest_preferred_stages is a string, convert to list
        if isinstance(pest_preferred_stages, str):
            preferred_stages = [s.strip() for s in pest_preferred_stages.split('/')]
        else:
            preferred_stages = []

        # Check for exact matches
        for stage in preferred_stages:
            if stage.lower() in current_stage.lower():
                return 1.0, stage

        # Check for partial matches (e.g., "Flowering" in "Pre-Flowering")
        for stage in preferred_stages:
            for part in stage.split('/'):
                part = part.strip().lower()
                if part and part in current_stage.lower():
                    return 0.7, stage  # Partial match

        # Handle general stages
        if "Elongation" in pest_preferred_stages and "growth" in current_stage.lower():
            return 0.5, "Growth (partial match)"

        return 0, "No match"

    def get_pest_probabilities(self, farm_data, pest_reference_data):
        """
        Estimate pest infestation probabilities based on how close
        farm parameters are to each pest's optimal conditions.
        """
        from datetime import datetime

        pest_probabilities = []
        skipped_pests = []

        crop_name = farm_data.get("crop_name")
        if not crop_name or crop_name not in pest_reference_data:
            return pest_probabilities

        current_date = datetime.now().strftime("%Y-%m-%d")
        crop_pests = pest_reference_data[crop_name]

        for pest_name, pest_info in crop_pests.items():
            temp_min = pest_info.get("Tmin")
            temp_max = pest_info.get("Tmax")

            try:
                if temp_min is not None:
                    temp_min = float(temp_min)
                if temp_max is not None:
                    temp_max = float(temp_max)
            except (ValueError, TypeError):
                temp_min = None
                temp_max = None

            if temp_min is not None and temp_max is None:
                temp_max = temp_min + 10
            elif temp_max is not None and temp_min is None:
                temp_min = temp_max - 10
            elif temp_min is None and temp_max is None:
                temp_min = 15
                temp_max = 35

            farm_temp = farm_data.get("max_temperature")
            try:
                if farm_temp is not None:
                    farm_temp = float(farm_temp)
            except (ValueError, TypeError):
                farm_temp = None

            temp_score = self.calculate_score(farm_temp, temp_min, temp_max)

            humidity_value = pest_info.get("Relative Humidity")

            try:
                if isinstance(humidity_value, str) and "%" in humidity_value:
                    humidity_value = float(humidity_value.strip("%")) / 100
                elif humidity_value is not None:
                    humidity_value = float(humidity_value)
            except (ValueError, TypeError):
                humidity_value = None

            if humidity_value is not None:
                humidity_min = max(0, humidity_value * 0.8)
                humidity_max = min(1, humidity_value * 1.2)
            else:
                humidity_min = 0.5
                humidity_max = 0.9

            farm_humidity = farm_data.get("humidity")
            try:
                if farm_humidity is not None:
                    farm_humidity = float(farm_humidity)
            except (ValueError, TypeError):
                farm_humidity = None

            humidity_score = self.calculate_score(farm_humidity, humidity_min, humidity_max)

            rainfall_score = 0.5

            current_stage = farm_data.get("crop_growth_stage")
            pest_stage = pest_info.get("Stage of Crop")
            growth_stage_score, matched_stage = self.growth_stage_match_score(current_stage, pest_stage)

            # Skip pests missing critical scores
            if any(score is None for score in [temp_score, humidity_score, growth_stage_score]):
                skipped_pests.append({
                    "pest_name": pest_name,
                    "reason": "Missing critical parameter scores",
                    "scores": {
                        "temperature": temp_score,
                        "humidity": humidity_score,
                        "growth_stage": growth_stage_score
                    }
                })
                continue

            scores = [temp_score, humidity_score, rainfall_score, growth_stage_score]
            valid_scores = [s for s in scores if s is not None]
            final_probability = sum(valid_scores) / len(valid_scores) if valid_scores else 0
            probability_percent = min(100, max(0, round(final_probability * 100, 1)))

            pest_probabilities.append({
                "pest_name": pest_name,
                "crop": crop_name,
                "probability": probability_percent,
                "date": current_date,
                "parameter_scores": {
                    "temperature": round(temp_score * 100, 1),
                    "humidity": round(humidity_score * 100, 1),
                    "rainfall": round(rainfall_score * 100, 1),
                    "growth_stage": round(growth_stage_score * 100, 1)
                },
                "growth_details": {
                    "current_stage": current_stage,
                    "pest_preferred_stage": pest_stage,
                    "matched_stage": matched_stage if growth_stage_score > 0 else "None",
                    "crop_age_days": farm_data.get("crop_age_days", 0)  # Include crop age in days
                }
            })

        pest_probabilities.sort(key=lambda x: x["probability"], reverse=True)
        return pest_probabilities

    def predict_pest_risks(self, merged_df):
        """
        Predict pest and disease risks for each date in the merged dataframe.
        Returns the top three pests with highest probabilities for each date.

        Args:
            merged_df: DataFrame containing both farm and weather data

        Returns:
            Dictionary with dates as keys and lists of top 3 pest predictions as values
        """
        import pandas as pd

        if merged_df.empty:
            return []

        pest_risks_by_date = {}

        for date, group in merged_df.groupby('date'):
            farm_data = {
                "crop_name": group['crop'].iloc[0],
                "max_temperature": float(group['maxtemp_c'].iloc[0]) if pd.notna(group['maxtemp_c'].iloc[0]) else None,
                "humidity": float(group['avghumidity'].iloc[0]) if pd.notna(group['avghumidity'].iloc[0]) else None,
                "rainfall": float(group['totalprecip_mm'].iloc[0]) if pd.notna(group['totalprecip_mm'].iloc[0]) else None,
                "crop_growth_stage": group['growth_stage'].iloc[0],
                "crop_age_days": int(group['crop_age_days'].iloc[0]) if pd.notna(group['crop_age_days'].iloc[0]) else 0
            }

            pest_probabilities = self.get_pest_probabilities(farm_data, self.library)

            if pest_probabilities:
                # Get top 3 pests for this date
                top_pests = pest_probabilities[:3]

                # Add the date to each pest
                for pest in top_pests:
                    pest["date"] = date
                    pest["crop_age_days"] = farm_data["crop_age_days"]  # Include crop age in days in the output

                # Add to results dictionary
                pest_risks_by_date[date] = top_pests
            else:
                # If no pests found, add an empty list for this date
                pest_risks_by_date[date] = []

        # Convert the dictionary to a flat list for backward compatibility
        pest_risks = []
        for date, pests in pest_risks_by_date.items():
            pest_risks.extend(pests)

        return pest_risks

    def run_analysis(self, pest_disease_url, payload, sowing_date=None):
        data = self.fetch_weather_data(pest_disease_url, payload)
        if data.empty:
            return []

        # Process the data while passing sowing_date
        farm_df, weather_df, merged_df = self.process_data(data, sowing_date)

        return self.predict_pest_risks(merged_df)

def upload_pest_disease_mongo(pest_disease_url, mobile, farm_name, pest_disease_data):
    """
    Uploads pest and disease data to MongoDB via an API endpoint.

    :param pest_disease_url: API URL for uploading data
    :param mobile: Mobile number associated with the farm
    :param farm_name: Name of the farm
    :param pest_disease_data: Dictionary containing pest/disease name and probability
    :return: API response JSON or error message
    """
    payload = {
        "mobile": mobile,
        "farm_name": farm_name,
        "pest_disease_data": pest_disease_data
    }

    try:
        response = requests.post(pest_disease_url, json=payload, headers={"Content-Type": "application/json"})
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error uploading data: {e}")
        return {"error": str(e)}



In [ ]:
mobile = "9638527411"
farm_name = "MD_Test_24_06_heatmap"
file_path = 'sentinelhub.id'
start_date, end_date, polygon, crop_season, config, area, crop_type, country, start_sowing_date = process_farm_data(file_path, mobile, farm_name)
print(f"crop_type: {crop_type}")
print(f"Processed farm data: {start_date}, {end_date}")
print(f"Country: {country}")

# Step 2: Process all data
tiff_images, mean_ndvi_values, mean_ndmi_values, mean_psri_values, latest_timestamp, sowing_date, multi_index_stress_results = process_all_data(
    farm_name, start_date, end_date, polygon, config, mobile, crop_season, area
)
# if not tiff_images:  # Check if tiff_images is empty
#     logging.warning("No TIFF images were processed. Exiting further processing.")
#     return jsonify({"message": "No TIFF images available for processing."}), 200

print(f"Processed mean_ndvi_values: {mean_ndvi_values}")
print(f"Processed last_ndvi_date: {latest_timestamp}")
print(f"Processed tiff_images: {tiff_images}")
logging.info(f"Data processed successfully: {tiff_images}")
#organize downloaded tiff images
organized_images, index_data = process_tiff_images(tiff_images, polygon)
structured_images = structure_tiff_images_by_index(tiff_images)

# Crop parameters for sowing
crop_parameters = {
    "Rice": {"min_ndvi_threshold": 0.15, "rate_threshold": 0.025, "savgol_window": 9, "savgol_polyorder": 2},
    "Wheat": {"min_ndvi_threshold": 0.15, "rate_threshold": 0.02, "savgol_window": 7, "savgol_polyorder": 2},
    "Potato": {"min_ndvi_threshold": 0.2, "rate_threshold": 0.035, "savgol_window": 5, "savgol_polyorder": 2},
    "Mustard": {"min_ndvi_threshold": 0.15, "rate_threshold": 0.025, "savgol_window": 7, "savgol_polyorder": 2},
    "Sanghar_White_Onion": {"min_ndvi_threshold": 0.12, "rate_threshold": 0.03, "savgol_window": 5, "savgol_polyorder": 2},
    "White_Onion": {"min_ndvi_threshold": 0.13, "rate_threshold": 0.03, "savgol_window": 5, "savgol_polyorder": 2},
    "Red_Onion": {"min_ndvi_threshold": 0.12, "rate_threshold": 0.03, "savgol_window": 5, "savgol_polyorder": 2},
    "Maize": {"min_ndvi_threshold": 0.15, "rate_threshold": 0.04, "savgol_window": 5, "savgol_polyorder": 2},
    "Cotton": {"min_ndvi_threshold": 0.2, "rate_threshold": 0.025, "savgol_window": 9, "savgol_polyorder": 2},
    "Gram": {"min_ndvi_threshold": 0.18, "rate_threshold": 0.03, "savgol_window": 7, "savgol_polyorder": 2},
    "Soyabean": {"min_ndvi_threshold": 0.2, "rate_threshold": 0.035, "savgol_window": 5, "savgol_polyorder": 2},
    "Barley": {"min_ndvi_threshold": 0.15, "rate_threshold": 0.025, "savgol_window": 7, "savgol_polyorder": 2},
    "Banana": {"min_ndvi_threshold": 0.25, "rate_threshold": 0.02, "savgol_window": 7, "savgol_polyorder": 2}
}

crop_harvest_parameters = {
    "india": {
        "Rice": {"decline_threshold": 0.15, "post_peak_buffer": 20, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 95, "min_maturity_days": 70, "harvested_threshold": 0.15},
        "Wheat": {"decline_threshold": 0.18, "post_peak_buffer": 18, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 95, "min_maturity_days": 80, "harvested_threshold": 0.15},
        "Potato": {"decline_threshold": 0.25, "post_peak_buffer": 12, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 90, "min_maturity_days": 65, "harvested_threshold": 0.15},
        "Mustard": {"decline_threshold": 0.15, "post_peak_buffer": 15, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 140, "min_maturity_days": 85, "harvested_threshold": 0.15},
        "Sanghar_White_Onion": {"decline_threshold": 0.15, "post_peak_buffer": 10, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 120, "min_maturity_days": 90, "harvested_threshold": 0.15},
        "White_Onion": {"decline_threshold": 0.15, "post_peak_buffer": 10, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 110, "min_maturity_days": 90, "harvested_threshold": 0.15},
        "Red_Onion": {"decline_threshold": 0.15, "post_peak_buffer": 10, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 120, "min_maturity_days": 90, "harvested_threshold": 0.15},
        "Maize": {"decline_threshold": 0.18, "post_peak_buffer": 15, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 95, "min_maturity_days": 75, "harvested_threshold": 0.15},
        "Cotton": {"decline_threshold": 0.2, "post_peak_buffer": 20, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 160, "min_maturity_days": 130, "harvested_threshold": 0.15},
        "Gram": {"decline_threshold": 0.19, "post_peak_buffer": 12, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 95, "min_maturity_days": 70, "harvested_threshold": 0.15},
        "Soyabean": {"decline_threshold": 0.18, "post_peak_buffer": 14, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 90, "min_maturity_days": 70, "harvested_threshold": 0.15},
        "Barley": {"decline_threshold": 0.16, "post_peak_buffer": 15, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 150, "min_maturity_days": 130, "harvested_threshold": 0.15},
        "Banana": {"decline_threshold": 0.15, "post_peak_buffer": 30, "savgol_window": 11, "savgol_polyorder": 2, "min_decline_days": 5, "harvest_days": 330, "min_maturity_days": 250, "harvested_threshold": 0.2}
    },
    # Slovakia-specific parameters
    "slovakia": {
        "Rice": {"decline_threshold": 0.15, "post_peak_buffer": 20, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 160, "min_maturity_days": 135, "harvested_threshold": 0.15},
        "Wheat": {"decline_threshold": 0.18, "post_peak_buffer": 18, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 270, "min_maturity_days": 250, "harvested_threshold": 0.15},
        "Maize": {"decline_threshold": 0.18, "post_peak_buffer": 15, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 224, "min_maturity_days": 200, "harvested_threshold": 0.15},
        "Soyabean": {"decline_threshold": 0.18, "post_peak_buffer": 14, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 156, "min_maturity_days": 120, "harvested_threshold": 0.15},
        "Barley": {"decline_threshold": 0.16, "post_peak_buffer": 15, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 150, "min_maturity_days": 130, "harvested_threshold": 0.15}
        },
    "Netherlands": {
        "White_Onion": {"decline_threshold": 0.15, "post_peak_buffer": 10, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 110, "min_maturity_days": 90, "harvested_threshold": 0.15},
        "Maize": {"decline_threshold": 0.18, "post_peak_buffer": 15, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 100, "min_maturity_days": 85, "harvested_threshold": 0.15}
        },
    # Philippines-specific parameters
    "philippines": {
        "Rice": {"decline_threshold": 0.15, "post_peak_buffer": 20, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 90, "min_maturity_days": 75, "harvested_threshold": 0.15},
        "Maize": {"decline_threshold": 0.18, "post_peak_buffer": 15, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 100, "min_maturity_days": 85, "harvested_threshold": 0.15}
        },
    # Egypt-specific parameters
    "egypt": {
        "Potato": {"decline_threshold": 0.16, "post_peak_buffer": 15, "savgol_window": 9, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 120, "min_maturity_days": 80, "harvested_threshold": 0.15},
        "Wheat": {"decline_threshold": 0.18, "post_peak_buffer": 15, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 150, "min_maturity_days": 130, "harvested_threshold": 0.15},
        "Rice": {"decline_threshold": 0.15, "post_peak_buffer": 18, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 150, "min_maturity_days": 135, "harvested_threshold": 0.15},
        "Maize": {"decline_threshold": 0.18, "post_peak_buffer": 15, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 110, "min_maturity_days": 95, "harvested_threshold": 0.15}
        },
    "default": {
        "Rice": {"decline_threshold": 0.15, "post_peak_buffer": 20, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 95, "min_maturity_days":70, "harvested_threshold": 0.15},
        "Wheat": {"decline_threshold": 0.18, "post_peak_buffer": 18, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 95, "min_maturity_days": 80, "harvested_threshold": 0.15},
        "Potato": {"decline_threshold": 0.25, "post_peak_buffer": 12, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 90, "min_maturity_days": 65, "harvested_threshold": 0.15},
        "Mustard": {"decline_threshold": 0.15, "post_peak_buffer": 15, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 140, "min_maturity_days": 85, "harvested_threshold": 0.15},
        "Sanghar_White_Onion": {"decline_threshold": 0.15, "post_peak_buffer": 10, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 120, "min_maturity_days": 90, "harvested_threshold": 0.15},
        "White_Onion": {"decline_threshold": 0.15, "post_peak_buffer": 10, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 110, "min_maturity_days": 90, "harvested_threshold": 0.15},
        "Red_Onion": {"decline_threshold": 0.15, "post_peak_buffer": 10, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 120, "min_maturity_days": 90, "harvested_threshold": 0.15},
        "Maize": {"decline_threshold": 0.18, "post_peak_buffer": 15, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 95, "min_maturity_days": 75, "harvested_threshold": 0.15},
        "Cotton": {"decline_threshold": 0.2, "post_peak_buffer": 20, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 160, "min_maturity_days": 130, "harvested_threshold": 0.15},
        "Gram": {"decline_threshold": 0.19, "post_peak_buffer": 12, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 95,"min_maturity_days": 70, "harvested_threshold": 0.15},
        "Soyabean": {"decline_threshold": 0.18, "post_peak_buffer": 14, "savgol_window": 5, "savgol_polyorder": 2, "min_decline_days": 3, "harvest_days": 90, "min_maturity_days": 70, "harvested_threshold": 0.15},
        "Barley": {"decline_threshold": 0.16, "post_peak_buffer": 15, "savgol_window": 7, "savgol_polyorder": 2, "min_decline_days": 2, "harvest_days": 150, "min_maturity_days": 130, "harvested_threshold": 0.15}
        },
    }

# Analyze the crop cycle
# Step 5: Analyze the crop cycle
sowing_date, harvest_window_start, harvest_window_end, harvested_date = analyze_crop_cycle(
    index_data, crop_type, crop_parameters, sowing_date,
    crop_harvest_parameters, country, mean_ndvi_values
)
print(f"sowing date: {sowing_date}")
print(f"harvest_window_start: {harvest_window_start}")
print(f"harvest_window_end: {harvest_window_end}")
print(f"harvested_date: {harvested_date}")

# Step 6: Process crop monitoring with cycle analysis
multi_index_stress_results_processed, growth_stages, crop_cycle_dates = process_crop_monitoring_with_cycle_analysis(
    tiff_images=organized_images,
    farm_boundary_gdf=polygon,
    crop_type=crop_type,
    crop_parameters=crop_parameters,
    crop_harvest_parameters=crop_harvest_parameters,
    country=country,
    sowing_date=sowing_date,
    harvest_start=harvest_window_start,
    harvest_end=harvest_window_end,
    harvested_date=harvested_date,
    index_data=index_data,
    multi_index_stress_results=multi_index_stress_results
)

# Transform stress data
# Transform stress data
stress_data_dict = transform_stress_data(multi_index_stress_results_processed, index_data, mean_ndvi_values)
print(f"stress_data_dict: {stress_data_dict}")

# Predict crop yield
estimator = SimpleYieldEstimator()
# crop_type = 'Onion'
yield_data = estimator.estimate_yield(
    crop_type=crop_type,
    stress_data=stress_data_dict,
    sowing_date=sowing_date,
    season=crop_season,
    country=country,
    harvest_date=harvested_date  # This might be None, ensure your estimator handles it
)



crop_type: Potato
Processed farm data: 2024-11-12 00:00:00, 2025-05-01 00:00:00
Country: India


Processed mean_ndvi_values: None
Processed last_ndvi_date: 2025-05-01
Processed tiff_images: {'NDVI_2025-04-25_to_2025-05-01': {'path': '/tmp/tmp3m4221mu/FIELD_MD_Test_24_06_heatmap_2025-04-25_to_2025-05-01_NDVI.tiff', 'data': array([[0.43055555, 0.4212963 , 0.32407406, ..., 0.22685185, 0.24074075,
        0.2685185 ],
       [0.6064815 , 0.6898148 , 0.6296296 , ..., 0.2361111 , 0.3148148 ,
        0.37962964],
       [0.6805556 , 0.7453704 , 0.7083333 , ..., 0.4212963 , 0.5046296 ,
        0.5555556 ],
       ...,
       [0.4814815 , 0.43055555, 0.4722222 , ..., 0.49537036, 0.30092594,
        0.30555555],
       [0.3564815 , 0.25      , 0.4675926 , ..., 0.7222222 , 0.6527778 ,
        0.6064815 ],
       [0.21296297, 0.16203703, 0.4722222 , ..., 0.875     , 0.875     ,
        0.8611111 ]], dtype=float32), 'crs': 'EPSG:4326', 'transform': Affine(9.402793966890245e-05, 0.0, 73.98502518854986,
       0.0, -9.119534666619832e-05, 18.4797756443543), 'height': 86, 'width': 123, 'index_nam

sowing date: 2025-02-03 00:00:00
harvest_window_start: 2025-05-04 00:00:00
harvest_window_end: 2025-05-11 00:00:00
harvested_date: None
Filtering images for NDVI: Found 24 images.
Extracted 24 images for index NDVI
Filtering images for NDMI: Found 24 images.
Extracted 24 images for index NDMI
Filtering images for PSRI: Found 24 images.
Extracted 24 images for index PSRI
Filtering images for MSAVI: Found 24 images.
Extracted 24 images for index MSAVI
Filtering images for NDRE: Found 24 images.
Extracted 24 images for index NDRE
Filtering images for GCI: Found 24 images.
Extracted 24 images for index GCI
Filtering images for SIPI: Found 24 images.
Extracted 24 images for index SIPI
Standardized images structure:
  NDVI: 24 entries
  NDMI: 24 entries
  PSRI: 24 entries
  MSAVI: 24 entries
  NDRE: 24 entries
  GCI: 24 entries
  SIPI: 24 entries
NDVI timestamps extracted: ['2025-04-25_to_2025-05-01', '2025-04-18_to_2025-04-24', '2025-04-11_to_2025-04-17', '2025-04-04_to_2025-04-10', '2025-0

In [ ]:
# Step 9: Upload to MongoDB
url = "https://3mpqory1se.execute-api.ap-south-1.amazonaws.com/prod/saveTiff"
response = None  # Initialize response variable

if structured_images and any(structured_images.values()):
    response = upload_to_mongo(
        url, mobile, farm_name, start_date, end_date, crop_season, area,
        latest_timestamp, index_data, structured_images,
        multi_index_stress_results_processed, sowing_date,
        harvest_window_start, harvest_window_end, harvested_date,
        yield_data, growth_stages
    )


start_date: 2024-11-12 00:00:00
Processing category: NDVI
Processing category: GCI
Processing category: MSAVI
Processing category: SIPI
Processing category: PSRI
Processing category: NDRE
Processing category: NDMI
Processing category: NDWI

MongoDB API Response for simplified payload:
Status Code: 413
Response Text: {"message": "Request Too Long"}
Failed to upload simplified data to MongoDB.


In [ ]:
growth_stages

{'2024-11-20_to_2024-11-26': 'Emergence',
 '2024-11-27_to_2024-12-03': 'Emergence',
 '2024-12-04_to_2024-12-10': 'Vegetative Growth',
 '2024-12-11_to_2024-12-17': 'Vegetative Growth',
 '2024-12-18_to_2024-12-24': 'Vegetative Growth',
 '2024-12-25_to_2024-12-31': 'Vegetative Growth',
 '2025-01-01_to_2025-01-07': 'Vegetative Growth',
 '2025-01-08_to_2025-01-14': 'Vegetative Growth',
 '2025-01-15_to_2025-01-21': 'Reproductive (Flowering/Fruit Set)',
 '2025-01-22_to_2025-01-28': 'Reproductive (Flowering/Fruit Set)',
 '2025-01-29_to_2025-02-04': 'Reproductive (Flowering/Fruit Set)',
 '2025-02-05_to_2025-02-11': 'Fruit Development',
 '2025-02-12_to_2025-02-18': 'Fruit Development',
 '2025-02-19_to_2025-02-25': 'Maturity / Senescence',
 '2025-02-26_to_2025-03-04': 'Maturity / Senescence'}

In [ ]:
multi_index_stress_results_processed

[{'timestamp': '2025-02-26_to_2025-03-04',
  'growth_stage': 'Maturity / Senescence',
  'stress_percentage': np.float64(80.15965496786046),
  'stress_type': 'Disease/Pest Stress',
  'stress_details': {'overall_stress_percentage': 80.15965496786046,
   'indices': {'NDVI': {'actual_mean': 0.18264928614755846,
     'expected_value': 0.4,
     'tolerance': 0.2,
     'deviation': 54.337678463110386,
     'stress_percentage': 88.67924528301887,
     'normalized_stress': 100.0,
     'combined_stress': 96.60377358490567,
     'weight': 0.1,
     'contribution': 9.660377358490567},
    'NDMI': {'actual_mean': 0.10401998635600596,
     'expected_value': 0.3,
     'tolerance': 0.25,
     'deviation': 65.32667121466467,
     'stress_percentage': 87.39495798319328,
     'normalized_stress': 100.0,
     'combined_stress': 96.21848739495798,
     'weight': 0.15,
     'contribution': 14.432773109243696},
    'PSRI': {'actual_mean': 0.8275174463496489,
     'expected_value': 0.65,
     'tolerance': 0.2

In [ ]:
# Predict crop yield
estimator = SimpleYieldEstimator()
yield_data = estimator.estimate_yield(
    crop_type=crop_type,
    stress_data=stress_data_dict,
    sowing_date=sowing_date,
    season=crop_season,
    country=country,
    harvest_date=harvested_date
)

Original date_value: Timestamp('2025-02-26 00:00:00')


In [ ]:
yield_data

{'estimated_yield': np.float64(26.35),
 'confidence_score': 96.0,
 'metrics': {'stress_impact': np.float64(1.019),
  'seasonal_factor': 1.0,
  'growth_stage_factor': 1.0,
  'days_after_sowing': 97,
  'harvest_date': '2025-02-26'}}

In [ ]:
multi_index_stress_results_processed

[{'timestamp': '2025-02-26_to_2025-03-04',
  'growth_stage': 'Maturity / Senescence',
  'stress_percentage': np.float64(89.87137758190754),
  'stress_type': 'Disease/Pest Stress',
  'stress_details': {'overall_stress_percentage': 89.87137758190754,
   'indices': {'NDVI': {'actual_mean': 0.18264928614755846,
     'expected_value': 0.4,
     'tolerance': 0.2,
     'deviation': 54.337678463110386,
     'stress_percentage': 88.67924528301887,
     'normalized_stress': 100.0,
     'combined_stress': 94.33962264150944,
     'weight': 0.1,
     'contribution': 9.433962264150944},
    'NDMI': {'actual_mean': 0.10401998635600596,
     'expected_value': 0.3,
     'tolerance': 0.25,
     'deviation': 65.32667121466467,
     'stress_percentage': 87.39495798319328,
     'normalized_stress': 100.0,
     'combined_stress': 93.69747899159664,
     'weight': 0.15,
     'contribution': 14.054621848739496},
    'PSRI': {'actual_mean': 0.8275174463496489,
     'expected_value': 0.65,
     'tolerance': 0.2

In [ ]:
growth_stages

{'2025-01-15_to_2025-01-21': 'Emergence',
 '2025-01-22_to_2025-01-28': 'Emergence',
 '2025-01-29_to_2025-02-04': 'Emergence',
 '2025-02-05_to_2025-02-11': 'Emergence',
 '2025-02-12_to_2025-02-18': 'Vegetative Growth',
 '2025-02-19_to_2025-02-25': 'Reproductive (Flowering/Fruit Set)',
 '2025-02-26_to_2025-03-04': 'Reproductive (Flowering/Fruit Set)',
 '2025-03-05_to_2025-03-11': 'Reproductive (Flowering/Fruit Set)',
 '2025-03-12_to_2025-03-18': 'Reproductive (Flowering/Fruit Set)',
 '2025-03-19_to_2025-03-25': 'Reproductive (Flowering/Fruit Set)',
 '2025-03-26_to_2025-04-01': 'Reproductive (Flowering/Fruit Set)',
 '2025-04-02_to_2025-04-08': 'Reproductive (Flowering/Fruit Set)',
 '2025-04-09_to_2025-04-15': 'Reproductive (Flowering/Fruit Set)',
 '2025-04-16_to_2025-04-22': 'Fruit Development',
 '2025-04-23_to_2025-04-29': 'Fruit Development'}

In [ ]:
harvest_end

Timestamp('2025-04-02 00:00:00')

In [ ]:
yield_data

{'estimated_yield': 18.0,
 'confidence_score': 96.0,
 'metrics': {'stress_impact': 0.8,
  'seasonal_factor': 1.0,
  'growth_stage_factor': 0.9,
  'days_after_sowing': 146,
  'harvest_date': '2025-04-02'}}

In [ ]:
# Process crop monitoring with cycle analysis
multi_index_stress_results_processed, growth_stages, crop_cycle_dates, ndvi_values, ndmi_values, psri_values, \
mean_msavi_values, mean_gci_values, mean_ndre_values, mean_sipi_values = process_crop_monitoring_with_cycle_analysis(
    tiff_images=organized_images,
    farm_boundary_gdf=polygon,
    crop_type=crop_type,
    crop_parameters=crop_parameters,
    crop_harvest_parameters=crop_harvest_parameters,
    country=country,
    sowing_date=sowing_date,
    mean_ndvi_values=mean_ndvi_values,
    mean_ndmi_values=mean_ndmi_values,
    mean_psri_values=mean_psri_values,
    multi_index_stress_results=multi_index_stress_results
)

Filtering images for NDVI: Found 28 images.
Filtering images for NDMI: Found 28 images.
Filtering images for PSRI: Found 28 images.
Filtering images for MSAVI: Found 28 images.
Filtering images for NDRE: Found 28 images.
Filtering images for GCI: Found 28 images.
Filtering images for SIPI: Found 28 images.


/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [ ]:
multi_index_stress_results_processed

[{'timestamp': '2025-04-04_to_2025-04-10',
  'growth_stage': 'Reproductive (Flowering/Fruit Set)',
  'stress_percentage': np.float64(76.85987633283023),
  'stress_type': 'Water Stress',
  'stress_details': {'overall_stress_percentage': 76.85987633283023,
   'indices': {'NDVI': {'actual_mean': 0.3358001828841541,
     'expected_value': 0.7,
     'tolerance': 0.15,
     'deviation': 52.028545302263694,
     'stress_percentage': 95.65217391304348,
     'normalized_stress': 100.0,
     'combined_stress': 97.82608695652175,
     'weight': 0.2,
     'contribution': 19.56521739130435},
    'NDMI': {'actual_mean': 0.010638298030863417,
     'expected_value': 0.5,
     'tolerance': 0.2,
     'deviation': 97.87234039382732,
     'stress_percentage': 100.0,
     'normalized_stress': 100.0,
     'combined_stress': 100.0,
     'weight': 0.2,
     'contribution': 20.0},
    'PSRI': {'actual_mean': 0.5420696308955233,
     'expected_value': 0.2,
     'tolerance': 0.25,
     'deviation': 171.034815447

In [ ]:

growth_stages

{'2025-01-10_to_2025-01-16': 'Emergence',
 '2025-01-17_to_2025-01-23': 'Emergence',
 '2025-01-24_to_2025-01-30': 'Vegetative Growth',
 '2025-01-31_to_2025-02-06': 'Vegetative Growth',
 '2025-02-07_to_2025-02-13': 'Vegetative Growth',
 '2025-02-14_to_2025-02-20': 'Vegetative Growth',
 '2025-02-21_to_2025-02-27': 'Vegetative Growth',
 '2025-02-28_to_2025-03-06': 'Vegetative Growth',
 '2025-03-07_to_2025-03-13': 'Vegetative Growth',
 '2025-03-14_to_2025-03-20': 'Vegetative Growth',
 '2025-03-21_to_2025-03-27': 'Reproductive (Flowering/Fruit Set)',
 '2025-03-28_to_2025-04-03': 'Reproductive (Flowering/Fruit Set)',
 '2025-04-04_to_2025-04-10': 'Reproductive (Flowering/Fruit Set)'}

In [ ]:
ndvi_values


{'2025-04-09_to_2025-04-15': 0.12214536051891867,
 '2025-04-16_to_2025-04-22': 0.13642116891006467,
 '2025-04-02_to_2025-04-08': 0.10017603704873777,
 '2025-03-26_to_2025-04-01': 0.10155085726167823,
 '2025-03-19_to_2025-03-25': 0.4634989752876954,
 '2025-03-12_to_2025-03-18': 0.6223815997115901,
 '2025-03-05_to_2025-03-11': 0.6775102456573581,
 '2025-02-26_to_2025-03-04': 0.6851618514686334,
 '2025-02-19_to_2025-02-25': 0.6500292744060032,
 '2025-02-12_to_2025-02-18': 0.6492461383831306,
 '2025-02-05_to_2025-02-11': 0.6306352450466547,
 '2025-01-29_to_2025-02-04': 0.53993101906581,
 '2025-01-22_to_2025-01-28': 0.5522830422784462,
 '2025-01-15_to_2025-01-21': 0.3853995902921821,
 '2025-01-08_to_2025-01-14': 0.17953096553263423,
 '2025-01-01_to_2025-01-07': 0.12123647561968594,
 '2024-12-25_to_2024-12-31': 0.10344727360261757,
 '2024-12-18_to_2024-12-24': 0.2184470298539725,
 '2024-12-11_to_2024-12-17': 0.20204918028511962,
 '2024-12-04_to_2024-12-10': 0.19027570710273184,
 '2024-11-27_

In [ ]:
multi_index_stress_results_processed

[{'timestamp': '2025-02-22_to_2025-02-28',
  'growth_stage': 'Maturity / Senescence',
  'stress_percentage': np.float64(73.52439692675837),
  'stress_type': 'Disease/Pest Stress',
  'stress_details': {'overall_stress_percentage': 73.52439692675837,
   'indices': {'NDVI': {'actual_mean': 0.22442942227308565,
     'expected_value': 0.4,
     'tolerance': 0.2,
     'deviation_percent': 43.89264443172859,
     'stress_pixel_percent': 86.53846153846155,
     'normalized_stress': 100.0,
     'combined_stress': 90.57692307692308,
     'weight': 0.1,
     'contribution': 9.057692307692308},
    'NDMI': {'actual_mean': 0.0,
     'expected_value': 0.3,
     'tolerance': 0.25,
     'deviation_percent': 100.0,
     'stress_pixel_percent': 100.0,
     'normalized_stress': 100.0,
     'combined_stress': 100.0,
     'weight': 0.15,
     'contribution': 15.0},
    'PSRI': {'actual_mean': 0.7696022697809067,
     'expected_value': 0.65,
     'tolerance': 0.2,
     'deviation_percent': 18.40034919706257

In [ ]:
growth_stages

{'2024-11-06_to_2024-11-12': 'Emergence',
 '2024-11-13_to_2024-11-19': 'Emergence',
 '2024-11-20_to_2024-11-26': 'Emergence',
 '2024-11-27_to_2024-12-03': 'Emergence',
 '2024-12-04_to_2024-12-10': 'Emergence',
 '2024-12-11_to_2024-12-17': 'Emergence',
 '2024-12-18_to_2024-12-24': 'Emergence',
 '2024-12-25_to_2024-12-31': 'Emergence',
 '2025-01-01_to_2025-01-07': 'Emergence',
 '2025-01-08_to_2025-01-14': 'Emergence',
 '2025-01-15_to_2025-01-21': 'Emergence',
 '2025-01-22_to_2025-01-28': 'Vegetative Growth',
 '2025-01-29_to_2025-02-04': 'Vegetative Growth',
 '2025-02-05_to_2025-02-11': 'Reproductive (Flowering/Fruit Set)',
 '2025-02-12_to_2025-02-18': 'Reproductive (Flowering/Fruit Set)',
 '2025-02-19_to_2025-02-25': 'Reproductive (Flowering/Fruit Set)',
 '2025-02-26_to_2025-03-04': 'Reproductive (Flowering/Fruit Set)',
 '2025-03-05_to_2025-03-11': 'Fruit Development',
 '2025-03-12_to_2025-03-18': 'Fruit Development',
 '2025-03-19_to_2025-03-25': 'Maturity / Senescence',
 '2025-03-26_to_

In [ ]:
sowing_date, harvest_start, harvest_end, harvested_date

(Timestamp('2024-11-24 00:00:00'),
 Timestamp('2025-03-24 00:00:00'),
 Timestamp('2025-03-31 00:00:00'),
 None)